In [3]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3

# required to enable non-blocking interaction:
%gui qt5

import sys
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from neuropy.utils.result_context import IdentifyingContext
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd
import re
from pathlib import Path
from datetime import datetime

from neuropy.utils.indexing_helpers import PandasHelpers
from pyphocorehelpers.indexing_helpers import partition_df
# Set the maximum number of columns to display
pd.set_option('display.max_columns', 100)

import IPython
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.notebook_helpers import NotebookCellExecutionLogger
from pyphocorehelpers.assertion_helpers import Assert

# Jupyter-lab enable printing for any line on its own (instead of just the last one in the cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

_notebook_path:Path = Path(IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())).resolve() # Finds the path of THIS notebook

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')

# Switch to the desired interactivity mode
plt.interactive(True)

import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from pyphocorehelpers.plotting.media_output_helpers import fig_to_clipboard
from pyphocorehelpers.Filesystem.path_helpers import file_uri_from_path, sanitize_filename_for_Windows
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, simple_path_display_widget
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_helper_save_figures, _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results
from pyphocorehelpers.assertion_helpers import Assert

# from ..PendingNotebookCode import plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import find_csv_files, find_HDF5_files, find_most_recent_files
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_histograms, plot_stacked_histograms

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DecoderDecodedEpochsResult
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_across_sessions_exported_files, _process_and_load_exported_file, _common_cleanup_operations

from pyphocorehelpers.programming_helpers import metadata_attributes
from pyphocorehelpers.function_helpers import function_attributes

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import build_session_t_delta, _new_process_csv_files, _old_process_csv_files

debug_print: bool = False
enable_neptune: bool = False

_TODAY_DAY_ONLY_DATE: str = "2024-09-25"
TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_Apogee"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_GL"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_Lab"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_rMBP"

print(f'TODAY_DAY_DATE: {TODAY_DAY_DATE}')

types.session_str: TypeAlias = str # a unique session identifier

if enable_neptune:
    import neptune # for logging progress and results
    from neptune.types import File
    from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import Neptuner, AutoValueConvertingNeptuneRun, set_environment_variables 

    ## Gets the notebook filepath for Neptune:
    import IPython
    from pyphocorehelpers.programming_helpers import IPythonHelpers
    notebook_filepath: str = IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())
    assert Path(notebook_filepath).resolve().exists(), f"found notebook filepath: '{notebook_filepath}' does not exist"
    # notebook_filepath

    neptune_kwargs = {'project':"commander.pho/PhoDibaLongShortAcrossSessions",
    'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}
            
    neptuner = Neptuner(project_name=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])


    if neptuner.run is None:
        neptuner.run = AutoValueConvertingNeptuneRun(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath])
        params = {"TODAY_DAY_DATE": TODAY_DAY_DATE, "run_workstation": "Apogee"}
        neptuner.run["parameters"] = params
        neptuner.outputs = neptuner.run['outputs']
        neptuner.figures = neptuner.outputs['figures']

    neptuner_run: AutoValueConvertingNeptuneRun = neptuner.run
    
    # run = neptune.init_run(source_files=["**/*.dvc"])

    # # Pre-execution dataframe view:
    # run["dataset/global_batch_run_progress_df"].upload(File.as_html(global_batch_run.to_dataframe(expand_context=True, good_only=False))) # "path/to/test_preds.csv"

else:
    # no neptune:
    neptuner = None    
    neptuner_run = None


known_bad_sessions = [IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44')]
known_bad_session_strs = [str(v.get_description()) for v in known_bad_sessions]
known_bad_session_strs

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
TODAY_DAY_DATE: 2024-09-25_Apogee


['kdiba_pin01_one_11-02_17-46-44']

### csv_files

In [5]:
## Load across session t_delta CSV, which contains the t_delta for each session:

## INPUTS: known_bad_session_strs,

cuttoff_date = datetime(2024, 9, 18)
# cuttoff_date = datetime(2024, 7, 1)
# cuttoff_date = datetime(2024, 5, 18)
# cuttoff_date = None

known_collected_outputs_paths = [Path(v).resolve() for v in ['/Users/pho/data/collected_outputs',
                                                            '/Volumes/SwapSSD/Data/collected_outputs', r"K:/scratch/collected_outputs", '/Users/pho/Dropbox (University of Michigan)/MED-DibaLabDropbox/Data/Pho/Outputs/output/collected_outputs', r'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/collected_outputs',
                                                            '/home/halechr/FastData/collected_outputs/', '/home/halechr/cloud/turbo/Data/Output/collected_outputs']]
collected_outputs_directory = find_first_extant_path(known_collected_outputs_paths)
assert collected_outputs_directory.exists(), f"collected_outputs_directory: {collected_outputs_directory} does not exist! Is the right computer's config commented out above?"
# fullwidth_path_widget(scripts_output_path, file_name_label='Scripts Output Path:')
print(f'collected_outputs_directory: {collected_outputs_directory}')

# _active_folder_widget = fullwidth_path_widget(collected_outputs_directory)
# display(_active_folder_widget)

# Create a 'figures' subfolder if it doesn't exist
figures_folder: Path = collected_outputs_directory.joinpath('figures').resolve()
figures_folder.mkdir(parents=False, exist_ok=True)
assert figures_folder.exists()
print(f'\tfigures_folder: {file_uri_from_path(figures_folder)}')

# Create an output path for the across session collected results (like the aggregate CSVs built from the individual session CSVs)
across_sessions_output_folder: Path = collected_outputs_directory.joinpath('../across_sessions').resolve()
across_sessions_output_folder.mkdir(parents=False, exist_ok=True)
assert across_sessions_output_folder.exists()
print(f'\tacross_sessions_output_folder: {file_uri_from_path(across_sessions_output_folder)}')

## sessions' t_delta:
# t_delta_csv_path = collected_outputs_directory.joinpath('../2024-01-18_GL_t_split_df.csv').resolve() # GL
t_delta_csv_path = collected_outputs_directory.joinpath('../2024-09-25_GL_t_split_df.csv').resolve()
# Assert.file_e
t_delta_df, t_delta_dict, (earliest_delta_aligned_t_start, latest_delta_aligned_t_end) = build_session_t_delta(t_delta_csv_path=t_delta_csv_path)

if neptuner_run is not None:
    _neptuner_run_parameters = dict(cuttoff_date=cuttoff_date, collected_outputs_directory=collected_outputs_directory.as_posix(), figures_folder=figures_folder.as_posix(),
                           across_sessions_output_folder=across_sessions_output_folder.as_posix(), t_delta_csv_path=t_delta_csv_path.as_posix())
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parameters/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

## Find the files:
csv_files = find_csv_files(collected_outputs_directory)
h5_files = find_HDF5_files(collected_outputs_directory)

csv_sessions, parsed_csv_files_df  = find_most_recent_files(found_session_export_paths=csv_files, cuttoff_date=cuttoff_date)
h5_sessions, parsed_h5_files_df = find_most_recent_files(found_session_export_paths=h5_files)

## OUTPUTS: csv_files, csv_sessions, parsed_csv_files_df
## OUTPUTS: h5_files, h5_sessions, parsed_h5_files_df

_neptuner_run_parameters = dict(csv_files=csv_files, h5_files=h5_files, csv_sessions=csv_sessions, h5_sessions=h5_sessions)

# #TODO 2024-03-02 12:12: - [ ] Could add weighted correlation if there is a dataframe for that and it's computed:
_df_raw_variable_names = ['simple_pf_pearson_merged_df', 'weighted_corr_merged_df']
_df_variables_names = ['laps_weighted_corr_merged_df', 'ripple_weighted_corr_merged_df', 'laps_simple_pf_pearson_merged_df', 'ripple_simple_pf_pearson_merged_df']

# # tbin_values_dict = {'laps': self.laps_decoding_time_bin_size, 'ripple': self.ripple_decoding_time_bin_size}
time_col_name_dict = {'laps': 'lap_start_t', 'ripple': 'ripple_start_t'} ## default should be 't_bin_center'

# fold older files:
# {'laps_marginals_df': 'lap_start_t', 'ripple_marginals_df': 'ripple_start_t', 'laps_time_bin_marginals_df':'t_bin_center', 'ripple_time_bin_marginals_df':'t_bin_center'}

if neptuner_run is not None:
    _neptuner_run_parameters = _neptuner_run_parameters | dict(earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
                                     t_delta_df=t_delta_df)
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

all_session_experiment_experience_csv_path = Path("./EXTERNAL/sessions_experiment_datetime_df.csv").resolve()
Assert.path_exists(all_session_experiment_experience_csv_path)
## NEW `parsed_csv_files_df1-based approach 2024-07-11 - 
## INPUTS: parsed_csv_files_df
dict_results, df_results, excluded_or_outdated_files_list = _new_process_csv_files(parsed_csv_files_df=parsed_csv_files_df, t_delta_dict=t_delta_dict, cuttoff_date=cuttoff_date, known_bad_session_strs=known_bad_session_strs, all_session_experiment_experience_csv_path=all_session_experiment_experience_csv_path, debug_print=False) # , known_bad_session_strs=known_bad_session_strs
(final_sessions_loaded_laps_dict, final_sessions_loaded_ripple_dict, final_sessions_loaded_laps_time_bin_dict, final_sessions_loaded_ripple_time_bin_dict, final_sessions_loaded_simple_pearson_laps_dict, final_sessions_loaded_simple_pearson_ripple_dict, final_sessions_loaded_laps_wcorr_dict, final_sessions_loaded_ripple_wcorr_dict, final_sessions_loaded_laps_all_scores_dict, final_sessions_loaded_ripple_all_scores_dict) = dict_results
(all_sessions_laps_df, all_sessions_ripple_df, all_sessions_laps_time_bin_df, all_sessions_ripple_time_bin_df, all_sessions_simple_pearson_laps_df, all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_laps_df, all_sessions_wcorr_ripple_df, all_sessions_all_scores_ripple_df) = df_results

collected_outputs_directory: K:\scratch\collected_outputs
	figures_folder: file:///K:/scratch/collected_outputs/figures
	across_sessions_output_folder: file:///K:/scratch/across_sessions
earliest_delta_aligned_t_start: -2057.225948, latest_delta_aligned_t_end: 1661.856002
ERR: Could not parse filename: "session_h5_files_copydict_2024-09-10_GL"
ERR: Could not parse filename: "session_h5_files_copydict_2024-09-11_GL"
WARN: curr_session_t_delta is None for session_str = "kdiba_vvp01_two_2006-4-28_12-38-13"
WARN: curr_session_t_delta is None for session_str = "kdiba_vvp01_two_2006-4-28_12-38-13"
WARN: curr_session_t_delta is None for session_str = "kdiba_vvp01_two_2006-4-28_12-38-13"
WARN: curr_session_t_delta is None for session_str = "kdiba_vvp01_two_2006-4-28_12-38-13"
WARN: curr_session_t_delta is None for session_str = "kdiba_vvp01_two_2006-4-28_12-38-13"
WARN: curr_session_t_delta is None for session_str = "kdiba_vvp01_two_2006-4-28_12-38-13"
WARN: curr_session_t_delta is None for se

In [ ]:
# missing sessions from `*_GL_t_split_df.csv`:
kdiba_vvp01_two_2006-4-16_18-47-52,
kdiba_vvp01_two_2006-4-17_12-52-15,
kdiba_vvp01_two_2006-4-25_13-20-55,
kdiba_vvp01_two_2006-4-28_12-38-13, 

In [6]:
all_sessions_laps_time_bin_df

,P_LR,P_RL,P_Long,P_Short,epoch_idx,t_bin_center,session_name,delta_aligned_start_t,time_bin_size,custom_replay_name,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,0.558248,0.441752,3.661615e-01,0.633839,0,26.944885,kdiba_vvp01_two_2006-4-16_18-47-52,-404.260071,0.025,,15,8,False
1,0.494060,0.505940,7.092039e-01,0.290796,0,26.969885,kdiba_vvp01_two_2006-4-16_18-47-52,-404.235071,0.025,,15,8,False
2,0.392746,0.607254,4.065333e-01,0.593467,0,26.994885,kdiba_vvp01_two_2006-4-16_18-47-52,-404.210071,0.025,,15,8,False
3,0.563907,0.436093,5.158699e-01,0.484130,0,27.019885,kdiba_vvp01_two_2006-4-16_18-47-52,-404.185071,0.025,,15,8,False
4,0.558248,0.441752,3.661615e-01,0.633839,0,27.044885,kdiba_vvp01_two_2006-4-16_18-47-52,-404.160071,0.025,,15,8,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
179998,0.017191,0.982809,3.960539e-10,1.000000,83,525.396746,kdiba_vvp01_two_2006-4-28_12-38-13,242.426141,0.058,,37,19,False
179999,0.004591,0.995409,3.612594e-03,0.996387,83,525.454746,kdiba_vvp01_two_2006-4-28_12-38-13,242.484141,0.058,,37,19,False
180000,0.009597,0.990403,2.178946e-04,0.999782,83,525.512746,kdiba_vvp01_two_2006-4-28_12-38-13,242.542141,0.058,,37,19,False
180001,0.425373,0.574627,6.556791e-02,0.934432,83,525.570746,kdiba_vvp01_two_2006-4-28_12-38-13,242.600141,0.058,,37,19,False


In [7]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_and_apply_session_experience_rank_csv

all_session_experiment_experience_csv_path = Path("./EXTERNAL/sessions_experiment_datetime_df.csv").resolve()
Assert.path_exists(all_session_experiment_experience_csv_path)
sessions_df, (experience_rank_map_dict, experience_orientation_rank_map_dict), _callback_add_df_columns = load_and_apply_session_experience_rank_csv(all_session_experiment_experience_csv_path, session_uid_str_sep='_')
df_results = [_callback_add_df_columns(a_df, session_id_column_name='session_name') for a_df in df_results]
all_sessions_all_scores_ripple_df: pd.DataFrame = _callback_add_df_columns(all_sessions_all_scores_ripple_df, session_id_column_name='session_name')


# all_sessions_all_scores_ripple_df
# all_sessions_all_scores_ripple_df['session_name']
sessions_df

,Unnamed: 0,format_name,animal,exper_name,session_name,path,session_datetime,experience_rank,experience_orientation_rank,session_uid
0,31,kdiba,vvp01,two,2006-4-09_16-40-54,W:/Data/KDIBA/vvp01/two/2006-4-09_16-40-54,2006-04-09 16:40:54,0,0,kdiba|vvp01|two|2006-4-09_16-40-54
1,12,kdiba,vvp01,one,2006-4-09_17-29-30,W:/Data/KDIBA/vvp01/one/2006-4-09_17-29-30,2006-04-09 17:29:30,1,0,kdiba|vvp01|one|2006-4-09_17-29-30
2,13,kdiba,vvp01,one,2006-4-10_12-25-50,W:/Data/KDIBA/vvp01/one/2006-4-10_12-25-50,2006-04-10 12:25:50,2,1,kdiba|vvp01|one|2006-4-10_12-25-50
3,32,kdiba,vvp01,two,2006-4-10_12-58-3,W:/Data/KDIBA/vvp01/two/2006-4-10_12-58-3,2006-04-10 12:58:03,3,1,kdiba|vvp01|two|2006-4-10_12-58-3
4,33,kdiba,vvp01,two,2006-4-10_19-11-57,W:/Data/KDIBA/vvp01/two/2006-4-10_19-11-57,2006-04-10 19:11:57,4,2,kdiba|vvp01|two|2006-4-10_19-11-57
...,...,...,...,...,...,...,...,...,...,...
63,59,kdiba,pin01,one,11-09_21-17-16,W:/Data/KDIBA/pin01/one/11-09_21-17-16,2006-11-09 21:17:16,11,11,kdiba|pin01|one|11-09_21-17-16
64,60,kdiba,pin01,one,11-09_22-4-5,W:/Data/KDIBA/pin01/one/11-09_22-4-5,2006-11-09 22:04:05,12,12,kdiba|pin01|one|11-09_22-4-5
65,61,kdiba,pin01,one,11-19_12-35-59,W:/Data/KDIBA/pin01/one/11-19_12-35-59,2006-11-19 12:35:59,13,13,kdiba|pin01|one|11-19_12-35-59
66,62,kdiba,pin01,one,11-19_13-2-0,W:/Data/KDIBA/pin01/one/11-19_13-2-0,2006-11-19 13:02:00,14,14,kdiba|pin01|one|11-19_13-2-0


In [8]:
# print(list(sessions_df.columns)) # ['Unnamed: 0', 'format_name', 'animal', 'exper_name', 'session_name', 'path', 'session_datetime', 'experience_rank', 'experience_orientation_rank', 'session_uid']

# sessions_df[['format_name', 'animal', 'exper_name', 'session_name']]

# ensure no repeats:
if sessions_df[['format_name', 'animal', 'exper_name', 'session_name']].duplicated().any():
    raise ValueError("Duplicate entries found in the specified columns.")

# sessions_df.drop_duplicates(subset=['format_name', 'animal', 'exper_name', 'session_name'], inplace=False)

if sessions_df[['session_uid']].duplicated().any():
    raise ValueError("Duplicate entries found in the session_uid columns.")


df: pd.DataFrame = deepcopy(sessions_df)

unique_animals = df['animal'].unique().tolist()
unique_exper_names = df['exper_name'].unique().tolist()
unique_session_names = df['session_name'].unique().tolist()

sessions_tree = {}
sessions_number_tree = {}


for animal in unique_animals:
    sessions_tree[animal] = list(unique_session_names)
    sessions_number_tree[animal] = len(list(unique_session_names))
    # unique_exper_names


    # for sess in unique_session_names:
# sessions_tree
sessions_number_tree


{'vvp01': 68, 'gor01': 68, 'pin01': 68}

In [9]:
# Performed 1 aggregation grouped on columns: 'animal', 'exper_name'
sessions_count_df = sessions_df.groupby(['animal', 'exper_name']).agg(session_name_count=('session_name', 'count')).reset_index()
# 	animal	exper_name	session_name_count
# 0	gor01	one	6
# 1	gor01	two	6
# 2	pin01	one	16
# 3	vvp01	one	19
# 4	vvp01	two	21

# sessions_count_df.to_clipboard() #['session_name_count']
sessions_count_df

,animal,exper_name,session_name_count
0,gor01,one,6
1,gor01,two,6
2,pin01,one,16
3,vvp01,one,19
4,vvp01,two,21


In [ ]:
all_sessions_all_scores_ripple_df


In [10]:
all_sessions_all_scores_ripple_df['time_bin_size'].unique() # array([0.01 , 0.02 , 0.025, 0.05 , 0.03 , 0.044, 0.058])
all_sessions_laps_time_bin_df['time_bin_size'].unique() # array([0.025, 0.03 , 0.044, 0.05 , 0.058])
all_sessions_laps_df['time_bin_size'].unique() # array([0.025, 0.03 , 0.044, 0.05 , 0.058])
all_sessions_wcorr_laps_df['time_bin_size'].unique() # array([0.025, 0.05 , 0.25 , 1.5  , 0.03 , 0.044, 0.058])
all_sessions_simple_pearson_laps_df['time_bin_size'].unique() # array([0.025, 0.05 , 0.25 , 1.5  , 0.03 , 0.044, 0.058])
all_sessions_ripple_df['time_bin_size'].unique() # array([0.025, 0.03 , 0.044, 0.05 , 0.058])
all_sessions_ripple_time_bin_df['time_bin_size'].unique() # array([0.025, 0.03 , 0.044, 0.05 , 0.058])

array([0.01 , 0.02 , 0.025, 0.05 , 0.03 , 0.044, 0.058])

array([0.025, 0.03 , 0.044, 0.05 , 0.058])

array([0.025, 0.03 , 0.044, 0.05 , 0.058])

array([0.025, 0.05 , 0.25 , 1.5  , 0.03 , 0.044, 0.058])

array([0.025, 0.05 , 0.25 , 1.5  , 0.03 , 0.044, 0.058])

array([0.025, 0.03 , 0.044, 0.05 , 0.058])

array([0.025, 0.03 , 0.044, 0.05 , 0.058])

In [ ]:
# all_sessions_ripple_time_bin_df

# h5_files

In [11]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_across_sessions_exported_h5_files

## INPUTS: h5_sessions, session_dict, cuttoff_date, known_bad_session_strs
parsed_h5_files_df, h5_contexts_paths_dict = load_across_sessions_exported_h5_files(collected_outputs_directory=collected_outputs_directory, cuttoff_date=cuttoff_date,
                                                                                    known_bad_session_strs=known_bad_session_strs)
h5_session_contexts = list(h5_contexts_paths_dict.keys())
included_h5_paths = list(h5_contexts_paths_dict.values())

if neptuner_run is not None:
    _neptuner_run_parameters = dict(parsed_h5_files_df=parsed_h5_files_df,
                                       )
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

parsed_h5_files_df
# h5_contexts_paths_dict


collected_outputs_directory: K:\scratch\collected_outputs


,export_datetime,session,custom_replay_name,file_type,decoding_time_bin_size_str,path
39,2024-09-25,2006-4-16_18-47-52,,time_bin_size_sweep_results,,K:\scratch\collected_outputs\2024-09-25_Apogee...
40,2024-09-25,2006-4-16_18-47-52-None,,time_bin_size_sweep_results,,K:\scratch\collected_outputs\2024-09-25_Apogee...
41,2024-09-25,2006-4-17_12-52-15-None,,time_bin_size_sweep_results,,K:\scratch\collected_outputs\2024-09-25_Apogee...
42,2024-09-25,2006-4-25_13-20-55,,time_bin_size_sweep_results,,K:\scratch\collected_outputs\2024-09-25_Apogee...
43,2024-09-25,2006-4-25_13-20-55-None,,time_bin_size_sweep_results,,K:\scratch\collected_outputs\2024-09-25_Apogee...
45,2024-09-25,2006-4-28_12-38-13-None,,time_bin_size_sweep_results,,K:\scratch\collected_outputs\2024-09-25_Apogee...
81,2024-09-18,2006-6-09_1-22-43-None,,time_bin_size_sweep_results,,K:\scratch\collected_outputs\2024-09-18_Apogee...


In [12]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations
from pyphocorehelpers.indexing_helpers import reorder_columns, reorder_columns_relative

included_session_contexts = deepcopy(h5_session_contexts)
included_h5_paths = deepcopy(included_h5_paths)
num_sessions = len(included_session_contexts)
(neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table), output_path_dicts = AcrossSessionTables.build_and_save_all_combined_tables(included_session_contexts, included_h5_paths,
                                                                                                                                                    override_output_parent_path=across_sessions_output_folder, output_path_suffix=f'{TODAY_DAY_DATE}',
                                                                                                                                                    should_restore_native_column_types=True, include_csv=True, include_pkl=True)


if neptuner_run is not None:
    _neptuner_run_parameters = dict(neuron_identities_table=neuron_identities_table, long_short_fr_indicies_analysis_table=long_short_fr_indicies_analysis_table, neuron_replay_stats_table=neuron_replay_stats_table,
                                       num_sessions=num_sessions)
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'parsed/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

    for output_name, a_paths_dict in output_path_dicts.items():
        for format_extension, an_output_path in a_paths_dict.items():
            neptuner_run[f"output_files/{format_extension}/{output_name}"].upload(an_output_path.resolve().as_posix())


# {'neuron_identities_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_identities_table.csv'),
#   '.pkl': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_identities_table.pkl')},
#  'long_short_fr_indicies_analysis_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/long_short_fr_indicies_analysis_table.csv'),
#   '.pkl': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/long_short_fr_indicies_analysis_table.pkl')},
#  'neuron_replay_stats_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_replay_stats_table.csv'),
#   '.pkl': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee/neuron_replay_stats_table.pkl')}}

output_path_dicts

## Move the "height" columns to the end
neuron_replay_stats_table = reorder_columns_relative(neuron_replay_stats_table, column_names=['neuron_uid', 'format_name', 'animal', 'exper_name', 'session_name', 'neuron_type', 'aclu', 'session_uid', 'session_datetime'], relative_mode='start')

neuron_replay_stats_table


failed for file path: K:\scratch\collected_outputs\2024-09-11_GL_2006-6-12_15-55-31_pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table``. Skipping.
failed for file path: K:\scratch\collected_outputs\2024-09-11_GL_11-03_12-3-25_pipeline_results.h5, table_key: /kdiba/pin01/one/11-03_12-3-25/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/pin01/one/11-03_12-3-25/neuron_identities/table``. Skipping.
concatenating dataframes from 13 of 15 files
failed for file path: K:\scratch\collected_outputs\2024-09-11_GL_2006-6-12_15-55-31_pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/global_computations/long_short_fr_indicies_analysis/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-12_15-55-31/global_computations/long_short_fr_indicies_analysis/tab

{'neuron_identities_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee/neuron_identities_table.csv'),
  '.pkl': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee/neuron_identities_table.pkl')},
 'long_short_fr_indicies_analysis_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee/long_short_fr_indicies_analysis_table.csv'),
  '.pkl': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee/long_short_fr_indicies_analysis_table.pkl')},
 'neuron_replay_stats_table': {'.csv': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee/neuron_replay_stats_table.csv'),
  '.pkl': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee/neuron_replay_stats_table.pkl')}}

,neuron_uid,format_name,animal,exper_name,session_name,neuron_type,aclu,session_uid,session_datetime,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,num_replays,long_num_replays,short_num_replays,custom_frs_index,is_rate_extrema,is_refined_exclusive,is_refined_LxC,is_refined_SxC,is_long_peak_left_cap,is_long_peak_right_cap,is_long_peak_either_cap,LS_pf_peak_x_diff,long_LR_pf2D_peak_x,long_LR_pf2D_peak_y,long_RL_pf2D_peak_x,long_RL_pf2D_peak_y,short_LR_pf2D_peak_x,short_LR_pf2D_peak_y,short_RL_pf2D_peak_x,short_RL_pf2D_peak_y,long_LR_pf1D_peak,long_RL_pf1D_peak,short_LR_pf1D_peak,short_RL_pf1D_peak,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,kdiba|gor01|one|2006-6-08_14-26-15|2,kdiba,gor01,one,2006-6-08_14-26-15,0,2,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,NaN,1,0.412424,0.115533,-0.296891,7.91274,NaN,NaN,4.162582,NaN,NaN,45,45,0,0.233430,False,False,False,False,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,False
1,kdiba|gor01|one|2006-6-08_14-26-15|3,kdiba,gor01,one,2006-6-08_14-26-15,0,3,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,141.385524,198.280871,1,0.274069,1.394595,1.120526,9.241991,8.536258,-0.705733,4.75803,4.965426,0.207397,91,23,68,-0.692958,True,False,False,False,0.0,0.0,0.0,-56.895346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,False
2,kdiba|gor01|one|2006-6-08_14-26-15|4,kdiba,gor01,one,2006-6-08_14-26-15,0,4,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,179.315755,2,0.119382,0.924896,0.805514,NaN,12.484801,NaN,NaN,6.704848,NaN,87,0,87,-0.982542,True,True,False,True,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,False
3,kdiba|gor01|one|2006-6-08_14-26-15|5,kdiba,gor01,one,2006-6-08_14-26-15,0,5,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,99.662271,202.073894,1,2.326046,2.012612,-0.313434,11.994904,11.315249,-0.679655,7.160475,6.66393,-0.496545,229,126,103,-0.035629,False,False,False,False,0.0,0.0,0.0,-102.411623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,False
4,kdiba|gor01|one|2006-6-08_14-26-15|6,kdiba,gor01,one,2006-6-08_14-26-15,2,6,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,NaN,1,25.389639,12.754636,-12.635004,105.811996,41.364962,-64.447034,65.600818,27.059799,-38.541019,602,345,257,0.099237,False,False,False,False,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,kdiba|pin01|one|fet11-01_12-58-54|28,kdiba,pin01,one,fet11-01_12-58-54,pyr,28,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,180.571251,176.844138,SHARED,1.227517,1.3169564137781686,0.08943915035797345,8.804706579928968,9.155474207048053,0.35076762711908493,5.0161119216745815,5.236215310413111,0.2201033887385293,181,100,81,-0.087569,False,False,False,False,NaN,NaN,NaN,NaN,175.339442,141.003729,186.412849,140.908855,176.615749,140.846341,142.356312,146.959402,55.712956,59.440069,130.255222,100.438316,74.542266,40.998246,0,0,True
802,kdiba|pin01|one|fet11-01_12-58-54|29,kdiba,pin01,one,fet11-01_12-58-54,pyr,29,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,57.576513,102.301872,SHARED,0.929991,0.8388929037677681,-0.0910981891459075,8.640241822933044,8.10684461459008,-0.5333972083429632,4.78511645792336,4.472868759178924,-0.3122476987444358,328,233,95,0.019260,False,False,False,False,NaN,NaN,NaN,NaN,81.552880,140.860160,48.385781,147.082681,162.371929,141.001542,82.035449,147.112137,122.800995,115.346769,152.617902,126.528109,29.816906,11.181340,0,0,True
803,kdiba|pin01|one|fet11-01_12-58-54|30,kdiba,pin01,one,fet11-01_12-58-54,

In [13]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults

# output_path_suffix: str = f'2024-09-11_GL'
output_path_suffix: str = f'2024-09-12'
inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=collected_outputs_directory, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')


global_batch_result_inst_fr_file_path: K:\scratch\collected_outputs\across_session_result_long_short_recomputed_inst_firing_rate_2024-09-12.pkl
Computing loaded session pickle file results : "K:/scratch/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-09-12.pkl"... done.
num_sessions: 13
num_sessions: 13


In [ ]:
across_session_inst_fr_computation

In [ ]:
across_sessions_instantaneous_frs_list

In [14]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults

# matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# output_path_suffix: str = f'{TODAY_DAY_DATE}'
# output_path_suffix: str = f'2024-09-11_GL'
output_path_suffix: str = f'2024-09-12'
print(F'output_path_suffix: {output_path_suffix}')
AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=collected_outputs_directory, output_path_suffix=output_path_suffix, plotting_enabled=False)

output_path_suffix: 2024-09-12
global_batch_result_inst_fr_file_path: K:\scratch\collected_outputs\across_session_result_long_short_recomputed_inst_firing_rate_2024-09-12.pkl
Computing loaded session pickle file results : "K:/scratch/collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-09-12.pkl"... done.
num_sessions: 13
num_sessions: 13
a_name: neuron_identities_table
loading K:\scratch\collected_outputs\2024-09-12\neuron_identities_table.pkl.
Computing loaded session pickle file results : "K:/scratch/collected_outputs/2024-09-12/neuron_identities_table.pkl"... done.
a_name: long_short_fr_indicies_analysis_table
loading K:\scratch\collected_outputs\2024-09-12\long_short_fr_indicies_analysis_table.pkl.
Computing loaded session pickle file results : "K:/scratch/collected_outputs/2024-09-12/long_short_fr_indicies_analysis_table.pkl"... done.
a_name: neuron_replay_stats_table
loading K:\scratch\collected_outputs\2024-09-12\neuron_replay_stats_table.pkl.
Com

{}

# Get filtered for a particular type of replay:

In [15]:
## filter by specific set of replays:
# dfs_list = (all_sessions_ripple_df, all_sessions_ripple_time_bin_df, all_sessions_simple_pearson_ripple_df, all_sessions_wcorr_ripple_df, all_sessions_all_scores_ripple_df)

replay_name: str = ''
# replay_name: str = 'withNewComputedReplays-qclu_[1, 2]-frateThresh_5.0' # 4307 rows
# replay_name: str = 'withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0' # 1417 rows, 1437 rows
# replay_name: str = 'withNormalComputedReplays-qclu_[1,2]-frateThresh_1.0' # 2802 rows, 2831 rows

time_bin_size: float = 0.025
# time_bin_size: float = 0.02
# time_bin_size: float = 0.01
filtered_all_sessions_ripple_df = deepcopy(all_sessions_ripple_df)[(all_sessions_ripple_df['custom_replay_name'] == replay_name) & (all_sessions_ripple_df['time_bin_size'] == time_bin_size)]
filtered_all_sessions_ripple_time_bin_df = deepcopy(all_sessions_ripple_time_bin_df)[(all_sessions_ripple_time_bin_df['custom_replay_name'] == replay_name) & (all_sessions_ripple_time_bin_df['time_bin_size'] == time_bin_size)]
filtered_all_sessions_simple_pearson_ripple_df = deepcopy(all_sessions_simple_pearson_ripple_df)[(all_sessions_simple_pearson_ripple_df['custom_replay_name'] == replay_name) & (all_sessions_simple_pearson_ripple_df['time_bin_size'] == time_bin_size)]
filtered_all_sessions_wcorr_ripple_df = deepcopy(all_sessions_wcorr_ripple_df)[(all_sessions_wcorr_ripple_df['custom_replay_name'] == replay_name) & (all_sessions_wcorr_ripple_df['time_bin_size'] == time_bin_size)]
filtered_all_sessions_all_scores_ripple_df = deepcopy(all_sessions_all_scores_ripple_df)[(all_sessions_all_scores_ripple_df['custom_replay_name'] == replay_name) & (all_sessions_all_scores_ripple_df['time_bin_size'] == time_bin_size)]
## OUTPUTS: filtered_all_sessions_ripple_df, filtered_all_sessions_ripple_time_bin_df, filtered_all_sessions_simple_pearson_ripple_df, filtered_all_sessions_wcorr_ripple_df, filtered_all_sessions_all_scores_ripple_df
# filtered_all_sessions_simple_pearson_ripple_df
filtered_all_sessions_all_scores_ripple_df


,start,stop,label,duration,is_user_annotated_epoch,is_valid_epoch,P_LR,P_RL,P_Long,P_Short,P_Long_LR,congruent_dir_bins_ratio_long_LR,coverage_long_LR,direction_change_bin_ratio_long_LR,integral_second_derivative_long_LR,jump_long_LR,longest_sequence_length_ratio_long_LR,pearsonr_long_LR,stddev_of_diff_long_LR,total_congruent_direction_change_long_LR,total_variation_long_LR,travel_long_LR,wcorr_long_LR,P_Long_RL,congruent_dir_bins_ratio_long_RL,coverage_long_RL,direction_change_bin_ratio_long_RL,integral_second_derivative_long_RL,jump_long_RL,longest_sequence_length_ratio_long_RL,pearsonr_long_RL,stddev_of_diff_long_RL,total_congruent_direction_change_long_RL,total_variation_long_RL,travel_long_RL,wcorr_long_RL,P_Short_LR,congruent_dir_bins_ratio_short_LR,coverage_short_LR,direction_change_bin_ratio_short_LR,integral_second_derivative_short_LR,jump_short_LR,longest_sequence_length_ratio_short_LR,pearsonr_short_LR,stddev_of_diff_short_LR,total_congruent_direction_change_short_LR,total_variation_short_LR,travel_short_LR,wcorr_short_LR,P_Short_RL,...,short_best_score,score_diff,long_best_velocity,short_best_velocity,velocity_diff,long_best_intercept,short_best_intercept,intercept_diff,long_best_speed,short_best_speed,speed_diff,long_best_wcorr,short_best_wcorr,wcorr_diff,long_best_pearsonr,short_best_pearsonr,pearsonr_diff,long_best_travel,short_best_travel,travel_diff,long_best_coverage,short_best_coverage,coverage_diff,long_best_jump,short_best_jump,jump_diff,long_best_longest_sequence_length_ratio,short_best_longest_sequence_length_ratio,longest_sequence_length_ratio_diff,long_best_direction_change_bin_ratio,short_best_direction_change_bin_ratio,direction_change_bin_ratio_diff,long_best_congruent_dir_bins_ratio,short_best_congruent_dir_bins_ratio,congruent_dir_bins_ratio_diff,long_best_total_congruent_direction_change,short_best_total_congruent_direction_change,total_congruent_direction_change_diff,long_best_total_variation,short_best_total_variation,total_variation_diff,long_best_integral_second_derivative,short_best_integral_second_derivative,integral_second_derivative_diff,long_best_stddev_of_diff,short_best_stddev_of_diff,stddev_of_diff_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
814,42.658077,42.690456,0,0.032379,False,False,0.003378,0.996622,0.968932,0.031068,0.003273,1.000000,0.385965,0.000000,0.000000,0.001919,1.000000,0.252205,0.000000,87.524594,87.524594,0.408993,0.724297,0.965659,1.000000,0.315789,0.000000,0.000000e+00,0.002503,1.000000,-0.125155,0.000000,114.162513,114.162513,0.533470,-0.515537,0.000105,1.000000,0.228070,0.000000,0.000000,0.000372,1.000000,0.178567,0.000000,11.416251,11.416251,0.079280,0.294705,0.030963,...,0.347231,-0.116434,-1189.192848908021,-713.5157093448119,475.67713956320915,-50524.16480057826,-30251.71966878143,20272.445131796834,1189.192848908021,713.5157093448119,475.67713956320915,0.724297,0.294705,0.429593,0.252205,0.178567,0.073638,0.408993,0.079280,0.329714,0.385965,0.228070,0.157895,0.001919,0.000372,0.001547,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,87.524594,11.416251,76.108342,87.524594,11.416251,76.108342,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,5,2,False
815,55.723809,55.897899,1,0.174090,False,False,0.684715,0.315285,0.480394,0.519606,0.328933,0.333333,0.403509,0.555556,128781.306536,0.003922,0.300000,0.152431,78.542924,251.157530,490.898808,0.254880,0.173317,0.151461,0.222222,0.333333,0.333333,1.710809e+05,0.004673,0.500000,0.066632,97.482741,388.152546,570.812567,0.296372,-0.141276,0.355782,0.444444,0.280702,0.333333,16001.725371,0.003472,0.400000,0.372032,44.918969,178.854604,277.795450,0.214348,0.093641,0.163824,...,0.209555,-0.032417,445.94731834045785,79.27952326052683,366.667795079931,24953.317501583566,4506.841761602025,20446.475739981543,445.94731834045785,79.27952326052683,366.667795079931,-0.141276,-0.079061,0.062216,0.066632,-0.007675,0.058957,0.296372,0.355290,-0.058918,0

In [ ]:
# all_sessions_laps_time_bin_df['time_bin_size'].unique() # [ nan, 0.25]

all_sessions_wcorr_laps_df['time_bin_size'].unique() # [ nan, 0.25]

# ([0.025, 0.01 , 0.05 ]



	   

## Session Enumerations

In [ ]:
dataframes = {
    'all_sessions_ripple_df': all_sessions_ripple_df,
    'all_sessions_ripple_time_bin_df': all_sessions_ripple_time_bin_df,
    'all_sessions_simple_pearson_ripple_df': all_sessions_simple_pearson_ripple_df,
    'all_sessions_wcorr_ripple_df': all_sessions_wcorr_ripple_df,
    'all_sessions_all_scores_ripple_df': all_sessions_all_scores_ripple_df
}

for name, df in dataframes.items():
    unique_time_bin_counts = df.groupby('custom_replay_name')['time_bin_size'].nunique()
    print(f"== '{name}':")
    print(unique_time_bin_counts)


In [ ]:
all_sessions_ripple_df['custom_replay_name'].unique()
all_sessions_ripple_time_bin_df['custom_replay_name'].unique()
all_sessions_simple_pearson_ripple_df['custom_replay_name'].unique()
all_sessions_all_scores_ripple_df['custom_replay_name'].unique() 

# ['', 'withOldestImportedReplays-qclu_XX-frateThresh_0.1',
# 'withNewComputedReplays-qclu_[1, 2]-frateThresh_5.0',
# 'withNewKamranExportedReplays-qclu_[1,2]-frateThresh_5.0',
# 'withNormalComputedReplays-qclu_[1,2]-frateThresh_1.0']


all_sessions_all_scores_ripple_df['custom_replay_name'].unique() ## OH no, only this one is missing values

In [ ]:
all_sessions_ripple_df

In [ ]:
# sess_names, replay_names = all_sessions_ripple_df['session_name'].str.split('__') # , maxsplit=1

for a_split in all_sessions_ripple_df['session_name'].str.split('__'):
	print(f'a_split: {a_split}')

	if len(a_split) > 1:
		print(f'a_split: {a_split}')


In [ ]:
all_sessions_ripple_time_bin_df['custom_replay_name'].unique()
# all_sessions_ripple_df['custom_replay_name'].unique()

In [ ]:
# pd.set_option('display.min_rows', 50)
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.show_dimensions', True)

with pd.option_context('display.min_rows', 50, 'display.show_dimensions', True):
    # max_elements
    display(parsed_csv_files_df)

# parsed_csv_files_df[parsed_csv_files_df['file_type'] == 'ripple_all_scores_merged_df']


In [ ]:
parsed_csv_files_df[parsed_csv_files_df['file_type'] == 'ripple_all_scores_merged_df']

# all_sessions_all_scores_ripple_df


In [ ]:
all_sessions_ripple_time_bin_df

In [ ]:
print(list(np.unique(parsed_csv_files_df.file_type))) # ['laps_marginals_df', 'laps_simple_pf_pearson_merged_df', 'laps_time_bin_marginals_df', 'laps_weighted_corr_merged_df', 'merged_complete_epoch_stats_df', 'ripple_all_scores_merged_df', 'ripple_marginals_df', 'ripple_simple_pf_pearson_merged_df', 'ripple_time_bin_marginals_df', 'ripple_weighted_corr_merged_df']

# Across Session CSV Outputs

In [17]:
display(parsed_csv_files_df)

across_sessions_parsed_csv_files_path = across_sessions_output_folder.joinpath(f'{TODAY_DAY_DATE}_parsed_csv_files_df.csv').resolve()
# parsed_csv_files_df.to_clipboard(excel=True)
parsed_csv_files_df.to_csv(across_sessions_parsed_csv_files_path)
display(fullwidth_path_widget(across_sessions_parsed_csv_files_path, file_name_label='across_sessions_parsed_csv_files_path:'))

,export_datetime,session,custom_replay_name,file_type,decoding_time_bin_size_str,path
573,2024-09-18 10:40:00,kdiba_gor01_one_2006-6-09_1-22-43,,laps_simple_pf_pearson_merged_df,0.025,K:\scratch\collected_outputs\2024-09-18_1040AM...
588,2024-09-18 17:40:00,kdiba_gor01_one_2006-6-09_1-22-43,,laps_simple_pf_pearson_merged_df,0.05,K:\scratch\collected_outputs\2024-09-18_0540PM...
630,2024-09-23 17:00:00,kdiba_gor01_one_2006-6-09_1-22-43,,laps_simple_pf_pearson_merged_df,0.25,K:\scratch\collected_outputs\2024-09-23_0500PM...
568,2024-09-18 06:40:00,kdiba_gor01_one_2006-6-09_1-22-43,,laps_simple_pf_pearson_merged_df,1.5,K:\scratch\collected_outputs\2024-09-18_0640AM...
656,2024-09-18 10:40:00,kdiba_gor01_one_2006-6-09_1-22-43,,laps_weighted_corr_merged_df,0.025,K:\scratch\collected_outputs\2024-09-18_1040AM...
...,...,...,...,...,...,...
3076,2024-09-25 07:00:00,kdiba_vvp01_two_2006-4-28_12-38-13,,ripple_weighted_corr_merged_df,0.025,K:\scratch\collected_outputs\2024-09-25_0700AM...
3078,2024-09-25 07:00:00,kdiba_vvp01_two_2006-4-28_12-38-13,,ripple_weighted_corr_merged_df,0.03,K:\scratch\collected_outputs\2024-09-25_0700AM...
3080,2024-09-25 07:00:00,kdiba_vvp01_two_2006-4-28_12-38-13,,ripple_weighted_corr_merged_df,0.044,K:\scratch\collected_outputs\2024-09-25_0700AM...
3082,2024-09-25 07:00:00,kdiba_vvp01_two_2006-4-28_12-38-13,,ripple_weighted_corr_merged_df,0.05,K:\scratch\collected_outputs\2024-09-25_0700AM...


Box(children=(Label(value='across_sessions_parsed_csv_files_path:', layout=Layout(width='auto')), HTML(value="…

In [18]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import export_across_session_CSVs

final_across_session_summary_CSVs_output_path = across_sessions_output_folder.resolve()
display(fullwidth_path_widget(final_across_session_summary_CSVs_output_path, file_name_label='final_across_session_summary_CSVs_output_path:'))
final_csv_export_paths = export_across_session_CSVs(final_output_path=final_across_session_summary_CSVs_output_path, TODAY_DAY_DATE=TODAY_DAY_DATE,
                                                    all_sessions_laps_df=all_sessions_laps_df,  all_sessions_ripple_df=all_sessions_ripple_df,  all_sessions_laps_time_bin_df=all_sessions_laps_time_bin_df,  all_sessions_ripple_time_bin_df=all_sessions_ripple_time_bin_df, 
                                                    all_sessions_simple_pearson_laps_df=all_sessions_simple_pearson_laps_df,  all_sessions_simple_pearson_ripple_df=all_sessions_simple_pearson_ripple_df,
                                                    all_sessions_all_scores_ripple_df=all_sessions_all_scores_ripple_df,  all_sessions_all_scores_laps_df=None,
                                                )

# final_csv_export_paths: {'AcrossSession_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Laps_per-Epoch.csv'),
#  'AcrossSession_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Ripple_per-Epoch.csv'),
#  'AcrossSession_Laps_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Laps_per-TimeBin.csv'),
#  'AcrossSession_Ripple_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_Ripple_per-TimeBin.csv'),
#  'AcrossSession_SimplePearson_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_SimplePearson_Laps_per-Epoch.csv'),
#  'AcrossSession_SimplePearson_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_SimplePearson_Ripple_per-Epoch.csv'),
#  'AcrossSession_AllScores_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-06-05_Apogee_AcrossSession_AllScores_Ripple_per-Epoch.csv')}

if neptuner_run is not None:
    _neptuner_run_parameters = dict(across_sessions_parsed_csv_files_path=across_sessions_parsed_csv_files_path.as_posix(), final_across_session_summary_CSVs_output_path=final_across_session_summary_CSVs_output_path.as_posix(),
                                       )
    for k, v in _neptuner_run_parameters.items():
        neptuner_run[f'output_files/{k}'] = v
    _neptuner_run_parameters = {} # reset after writing

    for k, v in final_csv_export_paths.items():
        neptuner_run[f"output_files/{k}"].upload(v.resolve().as_posix())
        
    neptuner_run.sync()
    
final_csv_export_paths

Box(children=(Label(value='final_across_session_summary_CSVs_output_path:', layout=Layout(width='auto')), HTML…

{'AcrossSession_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_Laps_per-Epoch.csv'),
 'AcrossSession_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_Ripple_per-Epoch.csv'),
 'AcrossSession_Laps_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_Laps_per-TimeBin.csv'),
 'AcrossSession_Ripple_per-TimeBin': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_Ripple_per-TimeBin.csv'),
 'AcrossSession_SimplePearson_Laps_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_SimplePearson_Laps_per-Epoch.csv'),
 'AcrossSession_SimplePearson_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_SimplePearson_Ripple_per-Epoch.csv'),
 'AcrossSession_AllScores_Ripple_per-Epoch': WindowsPath('K:/scratch/across_sessions/2024-09-25_Apogee_AcrossSession_AllScores_Ripple_per-Epoch.csv')}

In [ ]:
print(list(all_sessions_all_scores_ripple_df.columns)) # ['start', 'stop', 'label', 'duration', 'is_user_annotated_epoch', 'is_valid_epoch', 'P_LR', 'P_RL', 'P_Long', 'P_Short', 'P_Long_LR', 'score_long_LR', 'velocity_long_LR', 'intercept_long_LR', 'speed_long_LR', 'wcorr_long_LR', 'pearsonr_long_LR', 'travel_long_LR', 'coverage_long_LR', 'jump_long_LR', 'longest_sequence_length_ratio_long_LR', 'direction_change_bin_ratio_long_LR', 'congruent_dir_bins_ratio_long_LR', 'total_congruent_direction_change_long_LR', 'P_Long_RL', 'score_long_RL', 'velocity_long_RL', 'intercept_long_RL', 'speed_long_RL', 'wcorr_long_RL', 'pearsonr_long_RL', 'travel_long_RL', 'coverage_long_RL', 'jump_long_RL', 'longest_sequence_length_ratio_long_RL', 'direction_change_bin_ratio_long_RL', 'congruent_dir_bins_ratio_long_RL', 'total_congruent_direction_change_long_RL', 'P_Short_LR', 'score_short_LR', 'velocity_short_LR', 'intercept_short_LR', 'speed_short_LR', 'wcorr_short_LR', 'pearsonr_short_LR', 'travel_short_LR', 'coverage_short_LR', 'jump_short_LR', 'longest_sequence_length_ratio_short_LR', 'direction_change_bin_ratio_short_LR', 'congruent_dir_bins_ratio_short_LR', 'total_congruent_direction_change_short_LR', 'P_Short_RL', 'score_short_RL', 'velocity_short_RL', 'intercept_short_RL', 'speed_short_RL', 'wcorr_short_RL', 'pearsonr_short_RL', 'travel_short_RL', 'coverage_short_RL', 'jump_short_RL', 'longest_sequence_length_ratio_short_RL', 'direction_change_bin_ratio_short_RL', 'congruent_dir_bins_ratio_short_RL', 'total_congruent_direction_change_short_RL', 'ripple_start_t', 'long_best_travel', 'short_best_travel', 'travel_diff', 'long_best_coverage', 'short_best_coverage', 'coverage_diff', 'long_best_jump', 'short_best_jump', 'jump_diff', 'long_best_longest_sequence_length_ratio', 'short_best_longest_sequence_length_ratio', 'longest_sequence_length_ratio_diff', 'long_best_direction_change_bin_ratio', 'short_best_direction_change_bin_ratio', 'direction_change_bin_ratio_diff', 'long_best_congruent_dir_bins_ratio', 'short_best_congruent_dir_bins_ratio', 'congruent_dir_bins_ratio_diff', 'long_best_total_congruent_direction_change', 'short_best_total_congruent_direction_change', 'total_congruent_direction_change_diff', 'session_name', 'time_bin_size', 'delta_aligned_start_t']
# ['long_best_total_congruent_direction_change', 'short_best_total_congruent_direction_change']
['longest_sequence_length_ratio_diff', 'direction_change_bin_ratio_diff', 'congruent_dir_bins_ratio_diff', 'total_congruent_direction_change_diff']

In [ ]:
all_sessions_simple_pearson_ripple_df # 3138 rows × 24 columns

In [ ]:
all_sessions_simple_pearson_laps_df # 931 rows × 25 columns

In [ ]:
all_sessions_ripple_df # 17592 rows × 10 columns

In [ ]:
csv_files

In [ ]:

all_sessions_simple_pearson_ripple_df

# 2024-03-02 - Get only the user-annotated ripples:

In [19]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import _split_user_annotated_ripple_df

## Bump
# input_df = all_sessions_simple_pearson_ripple_df
# input_df = all_sessions_all_scores_ripple_df

all_sessions_all_scores_ripple_df, (valid_ripple_df, invalid_ripple_df), (user_approved_ripple_df, user_rejected_ripple_df) = _split_user_annotated_ripple_df(all_sessions_all_scores_ripple_df)

## 2024-03-14 - 'is_valid_epoch' column
# 'is_valid_epoch'
## OUTPUTS: valid_ripple_df, invalid_ripple_df, user_approved_ripple_df, user_rejected_ripple_df, (user_annotated_epoch_unique_session_names, unannotated_session_names)
user_approved_ripple_df

user_annotated_epoch_unique_session_names: ['kdiba_gor01_one_2006-6-09_1-22-43']
unannotated_session_names: {'kdiba_vvp01_two_2006-4-28_12-38-13', 'kdiba_vvp01_two_2006-4-17_12-52-15', 'kdiba_vvp01_two_2006-4-16_18-47-52', 'kdiba_vvp01_two_2006-4-25_13-20-55'}
n_input_df_rows: 1236
	 n_valid_df_rows: 408
	 n_invalid_df_rows: 828
n_input_df_rows: 408
	 n_user_approved_df_rows: 123
	 n_user_rejected_df_rows: 285


C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\SpecificResults\AcrossSessionResults.py:2009: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_sessions_user_annotated_ripple_df['pre_post_delta_category'] = 'post-delta'


,start,stop,label,duration,P_LR,P_RL,P_Long,P_Short,P_Long_LR,congruent_dir_bins_ratio_long_LR,coverage_long_LR,direction_change_bin_ratio_long_LR,integral_second_derivative_long_LR,jump_long_LR,longest_sequence_length_ratio_long_LR,pearsonr_long_LR,stddev_of_diff_long_LR,total_congruent_direction_change_long_LR,total_variation_long_LR,travel_long_LR,wcorr_long_LR,P_Long_RL,congruent_dir_bins_ratio_long_RL,coverage_long_RL,direction_change_bin_ratio_long_RL,integral_second_derivative_long_RL,jump_long_RL,longest_sequence_length_ratio_long_RL,pearsonr_long_RL,stddev_of_diff_long_RL,total_congruent_direction_change_long_RL,total_variation_long_RL,travel_long_RL,wcorr_long_RL,P_Short_LR,congruent_dir_bins_ratio_short_LR,coverage_short_LR,direction_change_bin_ratio_short_LR,integral_second_derivative_short_LR,jump_short_LR,longest_sequence_length_ratio_short_LR,pearsonr_short_LR,stddev_of_diff_short_LR,total_congruent_direction_change_short_LR,total_variation_short_LR,travel_short_LR,wcorr_short_LR,P_Short_RL,congruent_dir_bins_ratio_short_RL,coverage_short_RL,...,score_diff,long_best_velocity,short_best_velocity,velocity_diff,long_best_intercept,short_best_intercept,intercept_diff,long_best_speed,short_best_speed,speed_diff,long_best_wcorr,short_best_wcorr,wcorr_diff,long_best_pearsonr,short_best_pearsonr,pearsonr_diff,long_best_travel,short_best_travel,travel_diff,long_best_coverage,short_best_coverage,coverage_diff,long_best_jump,short_best_jump,jump_diff,long_best_longest_sequence_length_ratio,short_best_longest_sequence_length_ratio,longest_sequence_length_ratio_diff,long_best_direction_change_bin_ratio,short_best_direction_change_bin_ratio,direction_change_bin_ratio_diff,long_best_congruent_dir_bins_ratio,short_best_congruent_dir_bins_ratio,congruent_dir_bins_ratio_diff,long_best_total_congruent_direction_change,short_best_total_congruent_direction_change,total_congruent_direction_change_diff,long_best_total_variation,short_best_total_variation,total_variation_diff,long_best_integral_second_derivative,short_best_integral_second_derivative,integral_second_derivative_diff,long_best_stddev_of_diff,short_best_stddev_of_diff,stddev_of_diff_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category
0,105.400143,105.562560,8,0.162417,0.878831,1.211693e-01,7.703602e-01,0.229640,6.770162e-01,0.600000,0.421053,0.200000,67352.058552,0.002253,0.375000,0.842463,40.277543,273.990032,388.152546,0.120920,0.719119,9.334399e-02,0.266667,0.263158,0.466667,5.922231e+05,0.004673,0.250000,0.604015,126.905574,0.000000,1514.556012,0.471824,0.457207,0.201814,0.266667,0.385965,0.266667,71609.531184,0.003596,0.312500,0.775503,42.738296,209.297941,361.514626,0.167368,0.505433,2.782529e-02,0.466667,0.210526,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2,False,pre-delta
1,132.511389,132.791003,17,0.279613,0.245418,7.545819e-01,8.199614e-01,0.180039,2.012334e-01,0.461538,0.263158,0.423077,529041.658798,0.004256,0.222222,-0.754849,82.093407,742.056338,1286.230985,0.231170,-0.770705,6.187281e-01,0.500000,0.368421,0.230769,3.537323e+05,0.004506,0.444444,-0.835692,66.021519,521.342145,844.802600,0.151834,-0.855821,0.044185,0.461538,0.245614,0.269231,322814.897707,0.004712,0.592593,-0.621914,65.240118,532.758396,1057.905958,0.282560,-0.597772,1.358539e-01,0.423077,0.298246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2,False,pre-delta
2,149.959357,150.254392,28,0.295035,0.412687,5.873129e-01,6.138659e-01,0.386134,2.533345e-01,0.428571,0.438596,0.464286,192744.764421,0.002837,0.206897,-0.509764,49.357535,464.260888,860.024268,0.143529,-0.219065,3.605313e-01,0.464286,0.385965,0.392857,2.242559e+05,0.003087,0.206897,-0.467870,57.47

### 2024-02-29 - 4pm - Filter the events for those meeting wcorr criteria:


In [20]:
# df: pd.DataFrame = deepcopy(all_sessions_user_annotated_ripple_df)
df: pd.DataFrame = deepcopy(valid_ripple_df) # valid epochs, but not just those that the user approved
# df: pd.DataFrame = deepcopy(user_approved_ripple_df)

## INPUTS: df

min_wcorr_threshold: float = 0.33
min_wcorr_diff_threshold: float = 0.2

# is_included_large_wcorr_diff = np.any((df[['wcorr_abs_diff']].abs() > min_wcorr_diff_threshold), axis=1)
is_included_large_wcorr_diff = np.any((df[['wcorr_diff']].abs() > min_wcorr_diff_threshold), axis=1)
is_included_high_wcorr = np.any((df[['long_best_wcorr', 'short_best_wcorr']].abs() > min_wcorr_threshold), axis=1)

df = df[is_included_high_wcorr]
df

# wcorr_long_LR


,start,stop,label,duration,is_user_annotated_epoch,P_LR,P_RL,P_Long,P_Short,P_Long_LR,congruent_dir_bins_ratio_long_LR,coverage_long_LR,direction_change_bin_ratio_long_LR,integral_second_derivative_long_LR,jump_long_LR,longest_sequence_length_ratio_long_LR,pearsonr_long_LR,stddev_of_diff_long_LR,total_congruent_direction_change_long_LR,total_variation_long_LR,travel_long_LR,wcorr_long_LR,P_Long_RL,congruent_dir_bins_ratio_long_RL,coverage_long_RL,direction_change_bin_ratio_long_RL,integral_second_derivative_long_RL,jump_long_RL,longest_sequence_length_ratio_long_RL,pearsonr_long_RL,stddev_of_diff_long_RL,total_congruent_direction_change_long_RL,total_variation_long_RL,travel_long_RL,wcorr_long_RL,P_Short_LR,congruent_dir_bins_ratio_short_LR,coverage_short_LR,direction_change_bin_ratio_short_LR,integral_second_derivative_short_LR,jump_short_LR,longest_sequence_length_ratio_short_LR,pearsonr_short_LR,stddev_of_diff_short_LR,total_congruent_direction_change_short_LR,total_variation_short_LR,travel_short_LR,wcorr_short_LR,P_Short_RL,congruent_dir_bins_ratio_short_RL,...,score_diff,long_best_velocity,short_best_velocity,velocity_diff,long_best_intercept,short_best_intercept,intercept_diff,long_best_speed,short_best_speed,speed_diff,long_best_wcorr,short_best_wcorr,wcorr_diff,long_best_pearsonr,short_best_pearsonr,pearsonr_diff,long_best_travel,short_best_travel,travel_diff,long_best_coverage,short_best_coverage,coverage_diff,long_best_jump,short_best_jump,jump_diff,long_best_longest_sequence_length_ratio,short_best_longest_sequence_length_ratio,longest_sequence_length_ratio_diff,long_best_direction_change_bin_ratio,short_best_direction_change_bin_ratio,direction_change_bin_ratio_diff,long_best_congruent_dir_bins_ratio,short_best_congruent_dir_bins_ratio,congruent_dir_bins_ratio_diff,long_best_total_congruent_direction_change,short_best_total_congruent_direction_change,total_congruent_direction_change_diff,long_best_total_variation,short_best_total_variation,total_variation_diff,long_best_integral_second_derivative,short_best_integral_second_derivative,integral_second_derivative_diff,long_best_stddev_of_diff,short_best_stddev_of_diff,stddev_of_diff_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category
136,93.027055,93.465245,4,0.438190,False,0.554979,0.445021,0.507392,0.492608,0.281592,0.500000,0.368421,0.500000,637694.098126,0.004172,0.222222,0.589175,95.265151,939.938028,1723.853954,0.309823,0.507870,2.257998e-01,0.384615,0.333333,0.576923,1.017999e+06,0.004673,0.222222,0.627194,119.815227,1145.430552,2279.444853,0.409677,0.496553,0.273387,0.384615,0.333333,0.423077,240286.542152,0.005828,0.148148,0.474401,58.397783,528.952979,955.159696,0.255117,0.392682,0.219221,0.346154,...,0.005927,-0.0,-91.47637299291532,-91.47637299291532,248.27803970987515,-8330.728869222017,-8082.450829512142,0.0,91.47637299291532,-91.47637299291532,0.496553,0.377063,0.119489,0.627194,0.463406,0.163788,0.409677,0.278495,0.131183,0.333333,0.368421,-0.035088,0.004673,0.005828,-0.001155,0.222222,0.259259,-0.037037,0.576923,0.384615,0.192308,0.384615,0.346154,0.038462,1145.430552,570.812567,574.617985,2279.444853,1042.684290,1236.760563,1.017999e+06,264151.558813,753847.798758,119.815227,62.689511,57.125716,5,2,False,pre-delta
137,105.400143,105.562560,8,0.162417,True,0.997449,0.002551,0.827140,0.172860,0.825030,0.666667,0.368421,0.333333,17377.439317,0.001752,0.300000,0.842463,30.071090,190.270856,220.714193,0.114597,0.809407,2.109962e-03,0.555556,0.280702,0.333333,2.172035e+05,0.003672,0.700000,0.604015,85.581942,270.184615,540.369231,0.280566,0.546726,0.172419,0.333333,0.350877,0.444444,18000.130892,0.003596,0.300000,0.775503,39.551575,125.578765,197.881690,0.152686,0.602887,0.000441,0.333333,...,0.076904,-0.0,-0.0,0.0,42.78551541859196,84.64510370014965,-41.859588281557684,0.0,0.0,0.0,0.546726,0.398772,0.147955,0.604015,0.540470,0.063545,0.280566,0.284819,-0.004253,0.280702,0.245614,0.035088,0.003672,0.0

In [21]:
all_sessions_all_scores_ripple_df.time_bin_size.unique() # does not seem to return NaNs

array([0.01 , 0.025, 0.05 ])

In [22]:
all_sessions_ripple_time_bin_df.time_bin_size.unique()

array([0.025, 0.03 , 0.044, 0.05 , 0.058])

In [23]:
all_sessions_ripple_time_bin_df

,P_LR,P_RL,P_Long,P_Short,epoch_idx,t_bin_center,session_name,delta_aligned_start_t,time_bin_size,custom_replay_name,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,8.988220e-01,0.101178,9.289129e-02,0.907109,0,17.385972,kdiba_vvp01_two_2006-4-16_18-47-52,-413.818984,0.025,,15,8,False
1,4.101497e-01,0.589850,5.845717e-01,0.415428,0,17.410972,kdiba_vvp01_two_2006-4-16_18-47-52,-413.793984,0.025,,15,8,False
2,7.356991e-01,0.264301,3.434736e-01,0.656526,0,17.435972,kdiba_vvp01_two_2006-4-16_18-47-52,-413.768984,0.025,,15,8,False
3,5.582479e-01,0.441752,3.661615e-01,0.633839,0,17.460972,kdiba_vvp01_two_2006-4-16_18-47-52,-413.743984,0.025,,15,8,False
4,5.582479e-01,0.441752,3.661615e-01,0.633839,0,17.485972,kdiba_vvp01_two_2006-4-16_18-47-52,-413.718984,0.025,,15,8,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5362,8.997440e-05,0.999910,3.822106e-01,0.617789,32,462.443591,kdiba_vvp01_two_2006-4-28_12-38-13,179.472986,0.058,,37,19,False
5363,9.980698e-01,0.001930,8.580285e-02,0.914197,32,462.501591,kdiba_vvp01_two_2006-4-28_12-38-13,179.530986,0.058,,37,19,False
5364,1.533907e-02,0.984661,1.904271e-06,0.999998,32,462.559591,kdiba_vvp01_two_2006-4-28_12-38-13,179.588986,0.058,,37,19,False
5365,5.363598e-08,1.000000,1.309622e-10,1.000000,33,492.152734,kdiba_vvp01_two_2006-4-28_12-38-13,209.182129,0.058,,37,19,False


### Single-time bin size version:

In [28]:
from pyphoplacecellanalysis.SpecificResults.PendingNotebookCode import build_single_time_bin_size_dfs

# Select the periods:
# active_all_sessions_all_scores_ripple_df, (active_all_sessions_ripple_df, active_all_sessions_ripple_time_bin_df) = all_sessions_all_scores_ripple_df, (all_sessions_ripple_df, all_sessions_ripple_time_bin_df)
active_all_sessions_all_scores_ripple_df, (active_all_sessions_ripple_df, active_all_sessions_ripple_time_bin_df) = filtered_all_sessions_all_scores_ripple_df, (filtered_all_sessions_ripple_df, filtered_all_sessions_ripple_time_bin_df)

# target_time_bin_size: float = np.nan
# target_time_bin_size: float = 0.005 # 0.025 # 0.08222222
# target_time_bin_size: float = 0.02
target_time_bin_size: float = 0.025

## INPUTS: all_sessions_all_scores_ripple_df, (all_sessions_ripple_df, all_sessions_ripple_time_bin_df)
single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_time_bin_df = build_single_time_bin_size_dfs(all_sessions_all_scores_epochs_df=active_all_sessions_all_scores_ripple_df,
                                                                                                            all_sessions_epochs_df=active_all_sessions_ripple_df, all_sessions_epochs_time_bin_df=active_all_sessions_ripple_time_bin_df, target_time_bin_size=target_time_bin_size,
                                                                                                            included_columns = ['delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch'])

                                                                                                            
single_time_bin_size_all_sessions_laps_df, single_time_bin_size_all_sessions_laps_time_bin_df = build_single_time_bin_size_dfs(all_sessions_all_scores_epochs_df=all_sessions_simple_pearson_laps_df,
                                                                                                            all_sessions_epochs_df=all_sessions_laps_df, all_sessions_epochs_time_bin_df=all_sessions_laps_time_bin_df, target_time_bin_size=target_time_bin_size,
                                                                                                            included_columns = ['delta_aligned_start_t',])

## OUTPUTS: single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_time_bin_df
## OUTPUTS: single_time_bin_size_all_sessions_laps_df, single_time_bin_size_all_sessions_laps_time_bin_df 

single_time_bin_size_all_sessions_laps_df
single_time_bin_size_all_sessions_ripple_df

single_time_bin_size_all_sessions_laps_time_bin_df
single_time_bin_size_all_sessions_ripple_time_bin_df



all_sessions_ripple_df.time_bin_size.unique(): [0.025]
np.shape(single_time_bin_size_all_sessions_ripple_df): (160, 17)
all_sessions_ripple_time_bin_df.time_bin_size.unique(): [0.025]
np.shape(single_time_bin_size_all_sessions_ripple_time_bin_df): (1650, 14)
all_sessions_ripple_df.time_bin_size.unique(): [0.025 0.03  0.044 0.05  0.058]
np.shape(single_time_bin_size_all_sessions_ripple_df): (334, 15)
all_sessions_ripple_time_bin_df.time_bin_size.unique(): [0.025 0.03  0.044 0.05  0.058]
np.shape(single_time_bin_size_all_sessions_ripple_time_bin_df): (54087, 14)


C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\SpecificResults\PendingNotebookCode.py:585: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_df['pre_post_delta_category'][a_df['delta_aligned_start_t'] < 0.0] = 'pre-delta'
C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\SpecificResults\PendingNotebookCode.py:585: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_df['pre_post_delta_category'][a_df['delta_aligned_start_t'] < 0.0] = 'pre-delta'
C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\SpecificResults\PendingNoteb

,P_LR,P_RL,P_Long,P_Short,lap_idx,lap_start_t,session_name,delta_aligned_start_t,time_bin_size,custom_replay_name,epoch_idx,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category
420,0.592185,0.407815,0.500101,0.499899,0,26.625436,kdiba_vvp01_two_2006-4-17_12-52-15,-442.204127,0.025,,0,17,9,False,pre-delta
422,0.592185,0.407815,0.500101,0.499899,0,26.625436,kdiba_vvp01_two_2006-4-17_12-52-15,-442.204127,0.025,,0,17,9,False,pre-delta
423,0.592185,0.407815,0.500101,0.499899,0,26.625436,kdiba_vvp01_two_2006-4-17_12-52-15,-442.204127,0.025,,0,17,9,False,pre-delta
424,0.592185,0.407815,0.500101,0.499899,0,26.625436,kdiba_vvp01_two_2006-4-17_12-52-15,-442.204127,0.025,,0,17,9,False,pre-delta
425,0.592185,0.407815,0.500101,0.499899,0,26.625436,kdiba_vvp01_two_2006-4-17_12-52-15,-442.204127,0.025,,0,17,9,False,pre-delta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,0.294493,0.705507,0.332129,0.667871,81,796.835765,kdiba_vvp01_two_2006-4-17_12-52-15,328.006202,0.025,,81,17,9,False,post-delta
907,0.294493,0.705507,0.332129,0.667871,81,796.835765,kdiba_vvp01_two_2006-4-17_12-52-15,328.006202,0.025,,81,17,9,False,post-delta
908,0.294493,0.705507,0.332129,0.667871,81,796.835765,kdiba_vvp01_two_2006-4-17_12-52-15,328.006202,0.025,,81,17,9,False,post-delta
909,0.294493,0.705507,0.332129,0.667871,81,796.835765,kdiba_vvp01_two_2006-4-17_12-52-15,328.006202,0.025,,81,17,9,False,post-delta


,P_LR,P_RL,P_Long,P_Short,ripple_idx,ripple_start_t,session_name,delta_aligned_start_t,time_bin_size,custom_replay_name,epoch_idx,is_valid_epoch,is_user_annotated_epoch,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category
28,0.393685,0.606315,0.701398,0.298602,0,40.693504,kdiba_vvp01_two_2006-4-17_12-52-15,-428.136059,0.025,,0,True,False,17,9,False,pre-delta
29,0.552896,0.447104,0.738616,0.261384,1,41.541744,kdiba_vvp01_two_2006-4-17_12-52-15,-427.287819,0.025,,1,True,False,17,9,False,pre-delta
0,0.652029,0.347971,0.533545,0.466455,0,17.373472,kdiba_vvp01_two_2006-4-16_18-47-52,-413.831484,0.025,,0,True,False,15,8,False,pre-delta
1,0.355809,0.644191,0.583780,0.416220,1,31.583339,kdiba_vvp01_two_2006-4-16_18-47-52,-399.621617,0.025,,1,True,False,15,8,False,pre-delta
2,0.496533,0.503467,0.456332,0.543668,2,40.361144,kdiba_vvp01_two_2006-4-16_18-47-52,-390.843812,0.025,,2,True,False,15,8,False,pre-delta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0.608027,0.391973,0.454749,0.545251,60,781.433394,kdiba_vvp01_two_2006-4-17_12-52-15,312.603832,0.025,,60,True,False,17,9,False,post-delta
89,0.623387,0.376613,0.557336,0.442664,61,787.221654,kdiba_vvp01_two_2006-4-17_12-52-15,318.392091,0.025,,61,True,False,17,9,False,post-delta
90,0.455882,0.544118,0.752516,0.247484,62,787.737535,kdiba_vvp01_two_2006-4-17_12-52-15,318.907972,0.025,,62,True,False,17,9,False,post-delta
91,0.694377,0.305623,0.785500,0.214500,63,802.408570,kdiba_vvp01_two_2006-4-17_12-52-15,333.579007,0.025,,63,True,False,17,9,False,post-delta


,P_LR,P_RL,P_Long,P_Short,epoch_idx,t_bin_center,session_name,delta_aligned_start_t,time_bin_size,custom_replay_name,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category
42124,0.998582,0.001418,0.751533,2.484674e-01,0,10.590539,kdiba_vvp01_two_2006-4-28_12-38-13,-272.380066,0.025,,37,19,False,pre-delta
42125,0.965620,0.034380,0.991765,8.234630e-03,0,10.615539,kdiba_vvp01_two_2006-4-28_12-38-13,-272.355066,0.025,,37,19,False,pre-delta
42126,0.360505,0.639495,0.512896,4.871041e-01,0,10.640539,kdiba_vvp01_two_2006-4-28_12-38-13,-272.330066,0.025,,37,19,False,pre-delta
42127,0.993191,0.006809,0.999999,9.892677e-07,0,10.665539,kdiba_vvp01_two_2006-4-28_12-38-13,-272.305066,0.025,,37,19,False,pre-delta
42128,0.999978,0.000022,0.056420,9.435797e-01,0,10.690539,kdiba_vvp01_two_2006-4-28_12-38-13,-272.280066,0.025,,37,19,False,pre-delta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28976,0.120183,0.879817,0.409842,5.901583e-01,81,801.023265,kdiba_vvp01_two_2006-4-17_12-52-15,332.193702,0.025,,17,9,False,post-delta
28977,0.599020,0.400980,0.364249,6.357514e-01,81,801.048265,kdiba_vvp01_two_2006-4-17_12-52-15,332.218702,0.025,,17,9,False,post-delta
28978,0.599020,0.400980,0.364249,6.357514e-01,81,801.073265,kdiba_vvp01_two_2006-4-17_12-52-15,332.243702,0.025,,17,9,False,post-delta
28979,0.599020,0.400980,0.364249,6.357514e-01,81,801.098265,kdiba_vvp01_two_2006-4-17_12-52-15,332.268702,0.025,,17,9,False,post-delta


,P_LR,P_RL,P_Long,P_Short,epoch_idx,t_bin_center,session_name,delta_aligned_start_t,time_bin_size,custom_replay_name,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category,is_user_annotated_epoch,is_valid_epoch
1300,1.000000,0.000000e+00,0.000004,0.999996,0,1.939135,kdiba_vvp01_two_2006-4-28_12-38-13,-281.031470,0.025,,37,19,False,pre-delta,NaN,NaN
1301,1.000000,6.809208e-12,0.609809,0.390191,0,1.964135,kdiba_vvp01_two_2006-4-28_12-38-13,-281.006470,0.025,,37,19,False,pre-delta,NaN,NaN
1302,0.590110,4.098901e-01,0.508888,0.491112,0,1.989135,kdiba_vvp01_two_2006-4-28_12-38-13,-280.981470,0.025,,37,19,False,pre-delta,NaN,NaN
1303,0.590110,4.098901e-01,0.508888,0.491112,0,2.014135,kdiba_vvp01_two_2006-4-28_12-38-13,-280.956470,0.025,,37,19,False,pre-delta,NaN,NaN
1304,0.223041,7.769586e-01,0.420440,0.579560,0,2.039135,kdiba_vvp01_two_2006-4-28_12-38-13,-280.931470,0.025,,37,19,False,pre-delta,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,0.172227,8.277728e-01,0.937112,0.062888,64,806.509937,kdiba_vvp01_two_2006-4-17_12-52-15,337.680374,0.025,,17,9,False,post-delta,NaN,NaN
1017,0.599020,4.009803e-01,0.364249,0.635751,64,806.534937,kdiba_vvp01_two_2006-4-17_12-52-15,337.705374,0.025,,17,9,False,post-delta,NaN,NaN
1018,0.599020,4.009803e-01,0.364249,0.635751,64,806.559937,kdiba_vvp01_two_2006-4-17_12-52-15,337.730374,0.025,,17,9,False,post-delta,NaN,NaN
1019,0.029114,9.708864e-01,0.005976,0.994024,64,806.584937,kdiba_vvp01_two_2006-4-17_12-52-15,337.755374,0.025,,17,9,False,post-delta,NaN,NaN


In [29]:


# # Recover the important columns from the newer `all_sessions_all_scores_ripple_df`
# included_columns = ['delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch']  # Added 'delta_aligned_start_t' for the merge
# single_time_bin_size_all_sessions_ripple_df = pd.merge(single_time_bin_size_all_sessions_ripple_df, 
#                      all_sessions_all_scores_ripple_df[included_columns], 
#                      on='delta_aligned_start_t', 
#                      how='left')


# single_time_bin_size_all_sessions_ripple_df
## OUTPUTS: single_time_bin_size_all_sessions_ripple_df, single_time_bin_size_all_sessions_ripple_df
# single_time_bin_size_all_sessions_ripple_time_bin_df
all_sessions_simple_pearson_laps_df
# single_time_bin_size_all_sessions_laps_time_bin_df

,P_LR,P_RL,P_Long,P_Short,lap_idx,lap_start_t,P_Long_LR,P_Long_RL,P_Short_LR,P_Short_RL,most_likely_decoder_index,start,stop,label,duration,lap_id,lap_dir,long_LR_pf_peak_x_pearsonr,long_RL_pf_peak_x_pearsonr,short_LR_pf_peak_x_pearsonr,short_RL_pf_peak_x_pearsonr,best_decoder_index,session_name,time_bin_size,delta_aligned_start_t,custom_replay_name,epoch_idx,long_best_pf_peak_x_pearsonr,short_best_pf_peak_x_pearsonr,wcorr_long_LR,wcorr_long_RL,wcorr_short_LR,wcorr_short_RL,long_best_wcorr,short_best_wcorr,wcorr_abs_diff,pearsonr_abs_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category
0,0.404910,0.595090,4.948753e-01,0.505125,0,3.054774,2.003799e-01,2.944954e-01,0.204530,0.300595,3,3.054774,4.723224,0,1.668450,1,1,0.212939,0.390937,0.044009,0.245178,1,kdiba_gor01_one_2006-6-09_1-22-43,0.025,-1026.261835,,0,0.212939,0.044009,0.108035,0.167473,0.039434,0.066824,0.108035,0.039434,0.068600,0.168930,5,2,False,pre-delta
1,0.586423,0.413577,3.889457e-01,0.611054,1,6.356765,2.280866e-01,1.608591e-01,0.358336,0.252718,2,6.356765,8.926857,1,2.570092,2,0,0.237919,0.376852,0.263642,0.401269,3,kdiba_gor01_one_2006-6-09_1-22-43,0.025,-1022.959844,,1,0.376852,0.401269,0.077951,0.161955,0.083049,0.097254,0.161955,0.097254,0.064701,-0.024417,5,2,False,pre-delta
2,0.781240,0.218760,5.734195e-01,0.426580,2,45.195989,4.479781e-01,1.254414e-01,0.333262,0.093319,0,45.195989,50.869954,2,5.673965,3,1,-0.592649,-0.268660,-0.523199,-0.259213,0,kdiba_gor01_one_2006-6-09_1-22-43,0.025,-984.120620,,2,-0.268660,-0.259213,-0.426964,-0.166469,-0.317782,-0.053454,-0.166469,-0.053454,0.113015,0.009447,5,2,False,pre-delta
3,0.246983,0.753017,7.094099e-01,0.290590,3,65.785018,1.752124e-01,5.341974e-01,0.071771,0.218819,1,65.785018,72.223040,3,6.438022,4,0,0.618741,0.782472,0.497373,0.691085,1,kdiba_gor01_one_2006-6-09_1-22-43,0.025,-963.531591,,3,0.618741,0.497373,0.515332,0.680173,0.442161,0.541246,0.515332,0.442161,0.073171,0.121368,5,2,False,pre-delta
4,0.809280,0.190720,6.501582e-01,0.349842,4,84.303384,5.261598e-01,1.239984e-01,0.283120,0.066722,0,84.303384,91.044238,4,6.740854,5,1,-0.805770,-0.477042,-0.761186,-0.428059,0,kdiba_gor01_one_2006-6-09_1-22-43,0.025,-945.013225,,4,-0.477042,-0.428059,-0.682385,-0.379908,-0.535599,-0.250978,-0.379908,-0.250978,0.128929,0.048983,5,2,False,pre-delta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,NaN,NaN,NaN,NaN,79,492.297752,NaN,NaN,NaN,NaN,0,492.297752,495.335331,79,3.037579,80,0,0.076337,-0.562835,-0.526973,-0.830214,3,kdiba_vvp01_two_2006-4-28_12-38-13,1.500,209.327147,,79,-0.562835,-0.830214,0.034330,-0.382569,-0.160462,-0.570717,-0.382569,-0.570717,-0.188148,-0.267380,37,19,False,post-delta
2336,1.000000,0.000000,1.996995e-46,1.000000,80,497.002526,1.996995e-46,0.000000e+00,1.000000,0.000000,2,497.002526,503.842279,80,6.839753,81,1,0.439159,0.323124,0.640771,0.174394,2,kdiba_vvp01_two_2006-4-28_12-38-13,1.500,214.031921,,80,0.323124,0.174394,0.298349,0.147983,0.361828,0.102661,0.147983,0.102661,0.045321,0.148730,37,19,False,post-delta
2337,0.500000,0.500000,4.995963e-01,0.500404,81,507.146030,2.497982e-01,2.497982e-01,0.250202,0.250202,3,507.146030,510.348224,81,3.202194,82,0,0.034472,-0.355392,-0.409640,-0.736704,3,kdiba_vvp01_two_2006-4-28_12-38-13,1.500,224.175425,,81,0.034472,-0.409640,0.030629,-0.283696,-0.152085,-0.537883,0.030629,-0.152085,-0.121456,-0.375169,37,19,False,post-delta
2338,1.000000,0.000000,3.177038e-24,1.000000,82,511.952190,3.177038e-24,0.000000e+00,1.000000,0.000000,2,511.952190,515.787403,82,3.835213,83,1,0.629688,0.373851,0.838673,0.396841,2,kdiba_vvp01_two_2006-4-28_12-38-13,1.500,228.981585,,82,0.373851,0.396841,0.389308,0.134907,0.507818,0.111559,0.134907,0.111559,0.023348,-0.022989,37,19,False,post-delta


In [ ]:
# INPUTS: all_sessions_all_scores_ripple_df, (all_sessions_ripple_df, all_sessions_ripple_time_bin_df)
all_sessions_ripple_df

In [ ]:
all_sessions_ripple_time_bin_df

# 🟢🔜 2024-03-28 - AcrossSessionTable (PhoDibaPaper2023 formats) .h5 and figure exports

neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table: all tables of the same length, one entry per neuron

In [30]:
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionTables, AcrossSessionsResults, AcrossSessionsVisualizations
from neuropy.utils.mixins.HDF5_representable import HDF_Converter
from pyphoplacecellanalysis.General.Batch.runBatch import BatchResultDataframeAccessor

output_path_suffix: str = '2024-09-12'
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}.pkl'
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}_0.0009.pkl' # single time bin size
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}_0.0015.pkl' # single time bin size
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}_0.0025.pkl' # single time bin size
# inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}_0.025.pkl' # single time bin size
inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{output_path_suffix}_1000.0.pkl' # single time bin size

## INPUTS: included_session_contexts, included_h5_paths
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.build_all_known_tables(included_session_contexts, included_h5_paths, should_restore_native_column_types=True)

## different than load_all_combined_tables, which seems to work with `long_short_fr_indicies_analysis_table`
# graphics_output_dict |= AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=True)

## Load all across-session tables from the pickles:
output_path_suffix: str = f'{output_path_suffix}'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=collected_outputs_directory, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
# num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
# print(f'num_sessions: {num_sessions}')
num_sessions: int = len(long_short_fr_indicies_analysis_table['session_uid'].unique())
print(f'num_sessions: {num_sessions}')

inst_fr_output_load_filepath: Path = collected_outputs_directory.joinpath(inst_fr_output_filename).resolve() # single time bin size # non-instantaneous version
assert inst_fr_output_load_filepath.exists()
# inst_fr_output_filename: str = inst_fr_output_load_filepath.name
# across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=inst_fr_output_load_filepath.parent, inst_fr_output_filename=inst_fr_output_filename)

graphics_output_dict = AcrossSessionsResults.post_compute_all_sessions_processing(global_data_root_parent_path=collected_outputs_directory, output_path_suffix=output_path_suffix, plotting_enabled=True, output_override_path=Path('../../output'), inst_fr_output_filename=inst_fr_output_filename)

num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

# Convert byte strings to regular strings
neuron_replay_stats_table = neuron_replay_stats_table.applymap(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
neuron_replay_stats_table

failed for file path: K:\scratch\collected_outputs\2024-09-11_GL_2006-6-12_15-55-31_pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table``. Skipping.
failed for file path: K:\scratch\collected_outputs\2024-09-11_GL_11-03_12-3-25_pipeline_results.h5, table_key: /kdiba/pin01/one/11-03_12-3-25/neuron_identities/table. wth exception group ``/`` does not have a child named ``/kdiba/pin01/one/11-03_12-3-25/neuron_identities/table``. Skipping.
concatenating dataframes from 13 of 15 files
failed for file path: K:\scratch\collected_outputs\2024-09-11_GL_2006-6-12_15-55-31_pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/global_computations/long_short_fr_indicies_analysis/table. wth exception group ``/`` does not have a child named ``/kdiba/gor01/one/2006-6-12_15-55-31/global_computations/long_short_fr_indicies_analysis/tab

,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,num_replays,long_num_replays,short_num_replays,neuron_type,aclu,custom_frs_index,is_rate_extrema,is_refined_exclusive,is_refined_LxC,is_refined_SxC,is_long_peak_left_cap,is_long_peak_right_cap,is_long_peak_either_cap,LS_pf_peak_x_diff,session_uid,neuron_uid,long_LR_pf2D_peak_x,long_LR_pf2D_peak_y,long_RL_pf2D_peak_x,long_RL_pf2D_peak_y,short_LR_pf2D_peak_x,short_LR_pf2D_peak_y,short_RL_pf2D_peak_x,short_RL_pf2D_peak_y,long_LR_pf1D_peak,long_RL_pf1D_peak,short_LR_pf1D_peak,short_RL_pf1D_peak,peak_diff_LR_pf1D_peak,peak_diff_RL_pf1D_peak,session_datetime,session_experience_rank,session_experience_orientation_rank,is_novel_exposure
0,kdiba,gor01,one,2006-6-08_14-26-15,NaN,NaN,1,0.412424,0.115533,-0.296891,7.91274,NaN,NaN,4.162582,NaN,NaN,45,45,0,0,2,0.233430,False,False,False,False,0.0,0.0,0.0,NaN,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-06-08 14:26:15,2,1,False
1,kdiba,gor01,one,2006-6-08_14-26-15,141.385524,198.280871,1,0.274069,1.394595,1.120526,9.241991,8.536258,-0.705733,4.75803,4.965426,0.207397,91,23,68,0,3,-0.692958,True,False,False,False,0.0,0.0,0.0,-56.895346,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-06-08 14:26:15,2,1,False
2,kdiba,gor01,one,2006-6-08_14-26-15,NaN,179.315755,2,0.119382,0.924896,0.805514,NaN,12.484801,NaN,NaN,6.704848,NaN,87,0,87,0,4,-0.982542,True,True,False,True,0.0,0.0,0.0,NaN,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-06-08 14:26:15,2,1,False
3,kdiba,gor01,one,2006-6-08_14-26-15,99.662271,202.073894,1,2.326046,2.012612,-0.313434,11.994904,11.315249,-0.679655,7.160475,6.66393,-0.496545,229,126,103,0,5,-0.035629,False,False,False,False,0.0,0.0,0.0,-102.411623,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-06-08 14:26:15,2,1,False
4,kdiba,gor01,one,2006-6-08_14-26-15,NaN,NaN,1,25.389639,12.754636,-12.635004,105.811996,41.364962,-64.447034,65.600818,27.059799,-38.541019,602,345,257,2,6,0.099237,False,False,False,False,0.0,0.0,0.0,NaN,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006-06-08 14:26:15,2,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,kdiba,pin01,one,fet11-01_12-58-54,180.571251,176.844138,SHARED,1.227517,1.3169564137781686,0.08943915035797345,8.804706579928968,9.155474207048053,0.35076762711908493,5.0161119216745815,5.236215310413111,0.2201033887385293,181,100,81,pyr,28,-0.087569,False,False,False,False,NaN,NaN,NaN,NaN,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|28,175.339442,141.003729,186.412849,140.908855,176.615749,140.846341,142.356312,146.959402,55.712956,59.440069,130.255222,100.438316,74.542266,40.998246,2009-11-01 12:58:54,0,0,True
802,kdiba,pin01,one,fet11-01_12-58-54,57.576513,102.301872,SHARED,0.929991,0.8388929037677681,-0.0910981891459075,8.640241822933044,8.10684461459008,-0.5333972083429632,4.78511645792336,4.472868759178924,-0.3122476987444358,328,233,95,pyr,29,0.019260,False,False,False,False,NaN,NaN,NaN,NaN,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|29,81.552880,140.860160,48.385781,147.082681,162.371929,141.001542,82.035449,147.112137,122.800995,115.346769,152.617902,126.528109,29.816906,11.181340,2009-11-01 12:58:54,0,0,True
803,kdiba,pin01,one,fet11-01_12-58-54,120.937439,128.391665,SHARED,0.952187

### Figure Generation

In [31]:
import matplotlib
from neuropy.core.user_annotations import UserAnnotationsManager
from pyphocorehelpers.DataStructure.RenderPlots.MatplotLibRenderPlots import MatplotlibRenderPlots
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update
# matplotlib.use('Qt5Agg')
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsResults, AcrossSessionTables, AcrossSessionsVisualizations
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PaperFigureTwo

def _save_matplotlib_fig(matplotlib_output_container: MatplotlibRenderPlots):

    a_fig_context = matplotlib_output_container.context
    assert len(matplotlib_output_container.saved_figures) == 1
    a_saved_fig_path = matplotlib_output_container.saved_figures[0][0]
    assert a_saved_fig_path.exists()

    if neptuner_run is not None:
        a_full_figure_path_key: str = a_fig_context.get_description(separator='/', include_property_names=True, key_value_separator=':') # .replace(' ', '_')
        # a_full_figure_path_key: str = a_fig_context.get_description(separator=':', include_property_names=True, key_value_separator='|')
        print(f'a_full_figure_path_key: "{a_full_figure_path_key}"')
        # neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(a_fig)
        neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(a_saved_fig_path.as_posix())
        # neptuner.figures[f"{a_full_figure_path_key}"].upload(a_fig)
        

%matplotlib inline


# with matplotlib_configuration_update(is_interactive=False, backend='nbAgg'):
# 'collected_outputs/across_session_result_long_short_recomputed_inst_firing_rate_2024-06-11_GL.pkl'





In [ ]:
long_short_fr_indicies_analysis_table

In [ ]:
neuron_identities_table

In [ ]:
neuron_replay_stats_table

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import pho_stats_bar_graph_t_tests

LxC_Laps_T_result, SxC_Laps_T_result, LxC_Replay_T_result, SxC_Replay_T_result = pho_stats_bar_graph_t_tests(across_session_inst_fr_computation)

# LxC_Laps_T_result: TtestResult(statistic=5.550057784140024, pvalue=4.394229331160663e-05, df=16)
# SxC_Laps_T_result: TtestResult(statistic=-4.50982955925142, pvalue=0.001125880142367611, df=10)
# LxC_Replay_T_result: TtestResult(statistic=-0.4086778656072959, pvalue=0.6881937588138113, df=16)
# SxC_Replay_T_result: TtestResult(statistic=-3.551930809035679, pvalue=0.0052513298000637825, df=10)



# LxC_Replay_T_result is NOT p<0.05 significant (pvalue=0.6882)
# SxC_Replay_T_result IS NOT p<0.05 significant (pvalue=0.0052513298000637825)

In [ ]:
from pyphoplacecellanalysis.SpecificResults.PhoDiba2023Paper import PaperFigureTwo
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import AcrossSessionsVisualizations

# %matplotlib inline 
%matplotlib qt5
## Plotting:
graphics_output_dict = {}

# matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

## 2023-10-04 - Run `AcrossSessionsVisualizations` corresponding to the PhoDibaPaper2023 figures for all sessions
## Hacks the `PaperFigureTwo` and `InstantaneousSpikeRateGroupsComputation`
global_multi_session_context, _out_aggregate_fig_2 = AcrossSessionsVisualizations.across_sessions_bar_graphs(across_session_inst_fr_computation, num_sessions=num_sessions, enable_tiny_point_labels=False, enable_hover_labels=False, enabled_point_connection_lines=True, save_figure=False)
# plt.show()

In [ ]:
# across_session_inst_fr_computation
## Document `InstantaneousSpikeRateGroupsComputation`
from pyphocorehelpers.print_helpers import DocumentationFilePrinter
from pyphocorehelpers.print_helpers import print_keys_if_possible

doc_output_parent_folder = Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\DEVELOPER_NOTES\DataStructureDocumentation').resolve()
Assert.path_exists(doc_output_parent_folder)
doc_printer = DocumentationFilePrinter(doc_output_parent_folder=doc_output_parent_folder, doc_name='InstantaneousSpikeRateGroupsComputation')
doc_printer.save_documentation('InstantaneousSpikeRateGroupsComputation', across_session_inst_fr_computation, non_expanded_item_keys=['_reverse_cellID_index_map'])

In [ ]:
doc_printer = DocumentationFilePrinter(doc_output_parent_folder=doc_output_parent_folder, doc_name='SingleBarResult')
doc_printer.save_documentation('SingleBarResult', across_session_inst_fr_computation.Fig2_Replay_FR[0], non_expanded_item_keys=['_reverse_cellID_index_map'])
doc_printer.save_documentation('SingleBarResult', across_session_inst_fr_computation.Fig2_Replay_FR[1], non_expanded_item_keys=['_reverse_cellID_index_map'])
doc_printer.save_documentation('SingleBarResult', across_session_inst_fr_computation.Fig2_Replay_FR[2], non_expanded_item_keys=['_reverse_cellID_index_map'])
doc_printer.save_documentation('SingleBarResult', across_session_inst_fr_computation.Fig2_Replay_FR[3], non_expanded_item_keys=['_reverse_cellID_index_map'])

In [ ]:
across_session_inst_fr_computation.get_summary_dataframe()

In [ ]:
across_sessions_instantaneous_frs_list


In [ ]:
# print_keys_if_possible('InstantaneousSpikeRateGroupsComputation', across_session_inst_fr_computation, max_depth=3)



### 2024-09-10 - TODO: Filter out the datapoints from the bar plot corresponding to the novel vs. non-novel sessions

In [ ]:
import re
from typing import List, Tuple
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import ConciseSessionIdentifiers

# `across_session_inst_fr_computation.Fig2_Replay_FR`

novel_Fig2_Replay_FR = deepcopy(across_session_inst_fr_computation.Fig2_Replay_FR)
non_novel_Fig2_Replay_FR = deepcopy(across_session_inst_fr_computation.Fig2_Replay_FR)

# [0] and [1] both only use LxC_aclus
LxC_aclus = deepcopy(across_session_inst_fr_computation.Fig2_Replay_FR[0].LxC_aclus.tolist()) # ['a0s0_109', 'a0s1_3', 'a0s1_29', 'a0s1_103', 'a0s3_90', 'a0s4_91', 'a0s4_95', 'a1s1_23', 'a1s2_25', 'a1s3_14', 'a1s3_30', 'a1s3_32', 'a2s0_8', 'a2s0_27', 'a2s1_27']
_LxC_aclus_alt = deepcopy(across_session_inst_fr_computation.Fig2_Replay_FR[1].LxC_aclus.tolist()) # ['a0s0_109', 'a0s1_3', 'a0s1_29', 'a0s1_103', 'a0s3_90', 'a0s4_91', 'a0s4_95', 'a1s1_23', 'a1s2_25', 'a1s3_14', 'a1s3_30', 'a1s3_32', 'a2s0_8', 'a2s0_27', 'a2s1_27']
assert _LxC_aclus_alt == LxC_aclus

# [2] and [3] both only use SxC_aclus
SxC_aclus = deepcopy(across_session_inst_fr_computation.Fig2_Replay_FR[2].SxC_aclus.tolist()) # ['a0s0_109', 'a0s1_3', 'a0s1_29', 'a0s1_103', 'a0s3_90', 'a0s4_91', 'a0s4_95', 'a1s1_23', 'a1s2_25', 'a1s3_14', 'a1s3_30', 'a1s3_32', 'a2s0_8', 'a2s0_27', 'a2s1_27']
_SxC_aclus_alt = deepcopy(across_session_inst_fr_computation.Fig2_Replay_FR[3].SxC_aclus.tolist()) # ['a0s0_109', 'a0s1_3', 'a0s1_29', 'a0s1_103', 'a0s3_90', 'a0s4_91', 'a0s4_95', 'a1s1_23', 'a1s2_25', 'a1s3_14', 'a1s3_30', 'a1s3_32', 'a2s0_8', 'a2s0_27', 'a2s1_27']
assert _SxC_aclus_alt == SxC_aclus

parsed_LxC_aclus_df: pd.DataFrame = ConciseSessionIdentifiers.parse_concise_abbreviated_neuron_identifying_strings(LxC_aclus)
# parsed_LxC_aclus_df

parsed_SxC_aclus_df: pd.DataFrame = ConciseSessionIdentifiers.parse_concise_abbreviated_neuron_identifying_strings(SxC_aclus)
# parsed_SxC_aclus_df

# novel_LxC_indicies, non_novel_LxC_indicies = 
# partition_df(parsed_LxC_aclus_df, partitionColumn='is_session_novel')
# partition_df(parsed_SxC_aclus_df, partitionColumn='is_session_novel')

novel_LxC_indicies = np.where(parsed_LxC_aclus_df['is_session_novel'])[0]
non_novel_LxC_indicies = np.where(np.logical_not(parsed_LxC_aclus_df['is_session_novel']))[0]

(novel_LxC_indicies, non_novel_LxC_indicies)

novel_SxC_indicies = np.where(parsed_SxC_aclus_df['is_session_novel'])[0]
non_novel_SxC_indicies = np.where(np.logical_not(parsed_SxC_aclus_df['is_session_novel']))[0]

(novel_SxC_indicies, non_novel_SxC_indicies)

## OUTPUTS: (novel_LxC_indicies, non_novel_LxC_indicies), (novel_SxC_indicies, non_novel_SxC_indicies)

In [ ]:
curr_idx: int = 0
novel_Fig2_Replay_FR[curr_idx].LxC_aclus = novel_Fig2_Replay_FR[curr_idx].LxC_aclus[novel_LxC_indicies]
novel_Fig2_Replay_FR[curr_idx].values = novel_Fig2_Replay_FR[curr_idx].values[novel_LxC_indicies]
curr_idx: int = 1
novel_Fig2_Replay_FR[curr_idx].LxC_aclus = novel_Fig2_Replay_FR[curr_idx].LxC_aclus[novel_LxC_indicies]
novel_Fig2_Replay_FR[curr_idx].values = novel_Fig2_Replay_FR[curr_idx].values[novel_LxC_indicies]
curr_idx: int = 2
novel_Fig2_Replay_FR[curr_idx].SxC_aclus = novel_Fig2_Replay_FR[curr_idx].SxC_aclus[novel_SxC_indicies]
novel_Fig2_Replay_FR[curr_idx].values = novel_Fig2_Replay_FR[curr_idx].values[novel_SxC_indicies]
curr_idx: int = 3
novel_Fig2_Replay_FR[curr_idx].SxC_aclus = novel_Fig2_Replay_FR[curr_idx].SxC_aclus[novel_SxC_indicies]
novel_Fig2_Replay_FR[curr_idx].values = novel_Fig2_Replay_FR[curr_idx].values[novel_SxC_indicies]

across_session_inst_fr_computation.Fig2_Replay_FR = novel_Fig2_Replay_FR

In [ ]:
global_multi_session_context, _out_aggregate_fig_2 = AcrossSessionsVisualizations.across_sessions_bar_graphs(across_session_inst_fr_computation, num_sessions=num_sessions, enable_tiny_point_labels=True, enable_hover_labels=False, save_figure=False)

In [32]:
plt.close('all')

In [ ]:
## group by 'is_novel_exposure'
## for `long_short_fr_indicies_analysis_table`
is_novel_partitioned_dfs = dict(zip(*partition_df(long_short_fr_indicies_analysis_table, partitionColumn='is_novel_exposure')))
novel_session_uids = is_novel_partitioned_dfs[True]['session_uid'].unique().tolist()
non_novel_session_uids = is_novel_partitioned_dfs[False]['session_uid'].unique().tolist()
## build dicts:
novel_only_inst_fr_dict = {k:v for k, v in across_sessions_instantaneous_fr_dict.items() if k.get_description(separator='|') in novel_session_uids}
non_novel_inst_fr_dict = {k:v for k, v in across_sessions_instantaneous_fr_dict.items() if k.get_description(separator='|') not in novel_session_uids}
# novel_only_inst_fr_dict

In [ ]:
# all
# graphics_output_dict |= AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=False)
# split based on session novelty:
graphics_output_dict |= AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=is_novel_partitioned_dfs[True], num_sessions=len(novel_session_uids), save_figure=False)
graphics_output_dict |= AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=is_novel_partitioned_dfs[False], num_sessions=len(non_novel_session_uids), save_figure=False)


In [ ]:
graphics_output_dict |= AcrossSessionsVisualizations.across_sessions_long_and_short_firing_rate_replays_v_laps_figure(neuron_replay_stats_table=neuron_replay_stats_table, num_sessions=num_sessions, save_figure=True)


In [ ]:
## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=num_sessions) # some global context across all of the sessions, not sure what to put here.



In [ ]:
_restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# _restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=False, backend='AGG')

plt.show()


In [ ]:
plt.close('all')    

In [ ]:

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
matplotlib_output_container: MatplotlibRenderPlots = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=num_sessions, save_figure=False)

In [ ]:
from scipy.stats import pearsonr

long_short_fr_indicies_analysis_table


In [ ]:
duplicates = long_short_fr_indicies_analysis_table[['x_frs_index', 'y_frs_index']].duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")


In [ ]:
print(long_short_fr_indicies_analysis_table[['x_frs_index', 'y_frs_index']].std())


In [ ]:

# long_short_fr_indicies_analysis_table[['x_frs_index']].corrwith(long_short_fr_indicies_analysis_table['y_frs_index']) # long_short_fr_indicies_analysis_table[['x_frs_index']].corrwith(long_short_fr_indicies_analysis_table['y_frs_index'])
# Compute correlation and p-value
df_cleaned: pd.DataFrame = long_short_fr_indicies_analysis_table[['x_frs_index', 'y_frs_index']].dropna()
df_cleaned


In [ ]:
print(df_cleaned.shape)
corr, p_value = pearsonr(df_cleaned['x_frs_index'], df_cleaned['y_frs_index'])
print(f"Correlation: {corr}, P-value: {p_value}")
# Correlation: 0.48548405144431905, P-value: 1.0243398539233502e-48



In [ ]:
plt.close('all')

In [ ]:
graphics_output_dict.saved_figures[0][0] #.plot_data['saved_figures']

In [ ]:
from pyphocorehelpers.plotting.media_output_helpers import fig_to_clipboard

fig_to_clipboard(fig)


In [ ]:
# copy_image_to_clipboard(graphics_output_dict['figures'][0])
fig_to_clipboard(matplotlib_output_container.figures[0])

In [ ]:
matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_long_and_short_firing_rate_replays_v_laps_figure(neuron_replay_stats_table=neuron_replay_stats_table, num_sessions=num_sessions)

In [ ]:
_save_matplotlib_fig(graphics_output_dict)


In [ ]:
## Load the saved across-session results:
# Outputs: across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list, neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table

BATCH_DATE_TO_USE = f'2024-09-03'
inst_fr_output_filename: str = f'across_session_result_long_short_recomputed_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'

inst_fr_output_file = collected_outputs_directory.joinpath(inst_fr_output_filename).resolve()
Assert.path_exists(inst_fr_output_file)

across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=collected_outputs_directory, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import 

## Load all across-session tables from the pickles:
output_path_suffix: str = f'{BATCH_DATE_TO_USE}'
neuron_identities_table, long_short_fr_indicies_analysis_table, neuron_replay_stats_table = AcrossSessionTables.load_all_combined_tables(override_output_parent_path=collected_outputs_directory, output_path_suffix=output_path_suffix) # output_path_suffix=f'2023-10-04-GL-Recomp'
num_sessions = len(neuron_replay_stats_table.session_uid.unique().to_numpy())
print(f'num_sessions: {num_sessions}')
# neuron_replay_stats_table


In [ ]:
neptuner.stop()

## 2024-09-04 - Batch Output Files

In [33]:
from neuropy.core.user_annotations import UserAnnotationsManager
from pyphocorehelpers.Filesystem.metadata_helpers import FilesystemMetadata, get_file_metadata
from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, generate_copydict, copy_movedict, copy_file, save_copydict_to_text_file, read_copydict_from_text_file, invert_filedict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_str_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import check_output_h5_files, copy_files_in_filelist_to_dest
from pyphoplacecellanalysis.General.Batch.runBatch import ConcreteSessionFolder, BackupMethods

# a_batch_progress_df = included_session_batch_progress_df.copy()
included_session_contexts
h5_contexts_paths_dict
# h5_session_contexts

debug_print = False
known_global_data_root_parent_paths = [Path(r'/nfs/turbo/umms-kdiba/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data'), Path(r'/media/halechr/MAX/Data'), Path(r'/Volumes/MoverNew/data')] # , Path(r'/home/halechr/FastData'), Path(r'/home/halechr/turbo/Data'), Path(r'W:\Data'), Path(r'/home/halechr/cloud/turbo/Data')
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
## Build Pickle Path:
# Hardcoded included_session_contexts:
included_session_contexts = UserAnnotationsManager.get_hardcoded_good_sessions()
good_session_concrete_folders = ConcreteSessionFolder.build_concrete_session_folders(global_data_root_parent_path, included_session_contexts)

# Output Paths:
included_h5_paths = [get_file_str_if_file_exists(v.pipeline_results_h5) for v in good_session_concrete_folders]
# copy_dict = ConcreteSessionFolder.build_backup_copydict(good_session_concrete_folders, backup_mode=BackupMethods.RenameInSourceDirectory, only_include_file_types=['local_pkl', 'global_pkl'])
check_output_h5_files(included_h5_paths)

## OUTPUTS: included_h5_paths, included_session_contexts, good_session_concrete_folders

included_h5_paths


RESULT_DATE_TO_USE = '2024-09-04'
# RESULT_DATE_TO_USE = '2024-07-02'

# included_file_types_paths_dict = {'h5': included_h5_paths, 'recomputed_inst_fr_comps': []}
# included_file_types_paths_dict['recomputed_inst_fr_comps'] = [get_file_path_if_file_exists(v.output_folder.joinpath(f'{RESULT_DATE_TO_USE}_recomputed_inst_fr_comps_0.0005.h5').resolve()) for v in good_session_concrete_folders]
# included_file_types_paths_dict


[Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'one', session_name= '2006-6-08_14-26-15'),
 Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'one', session_name= '2006-6-09_1-22-43'),
 Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'one', session_name= '2006-6-12_15-55-31'),
 Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'two', session_name= '2006-6-07_16-40-19'),
 Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'two', session_name= '2006-6-08_21-16-25'),
 Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'two', session_name= '2006-6-09_22-24-40'),
 Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'two', session_name= '2006-6-12_16-53-46'),
 Context(format_name= 'kdiba', animal= 'vvp01', exper_name= 'one', session_name= '2006-4-09_17-29-30'),
 Context(format_name= 'kdiba', animal= 'vvp01', exper_name= 'one', session_name= '2006-4-10_12-25-50'),
 Context(format_name= 'kdiba', animal= 'vvp01', exper_name= 'two'

{Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'one', session_name= '2006-6-08_14-26-15'): WindowsPath('K:/scratch/collected_outputs/2024-09-11_GL_2006-6-08_14-26-15_pipeline_results.h5'),
 Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'one', session_name= '2006-6-09_1-22-43'): WindowsPath('K:/scratch/collected_outputs/2024-09-11_GL_2006-6-09_1-22-43_pipeline_results.h5'),
 Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'one', session_name= '2006-6-12_15-55-31'): WindowsPath('K:/scratch/collected_outputs/2024-09-11_GL_2006-6-12_15-55-31_pipeline_results.h5'),
 Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'two', session_name= '2006-6-07_16-40-19'): WindowsPath('K:/scratch/collected_outputs/2024-09-11_GL_2006-6-07_16-40-19_pipeline_results.h5'),
 Context(format_name= 'kdiba', animal= 'gor01', exper_name= 'two', session_name= '2006-6-08_21-16-25'): WindowsPath('K:/scratch/collected_outputs/2024-09-11_GL_2006-6-08_21-16-25_pipeline_result

WARN: file_size < 0.01 for W:\Data\KDIBA\gor01\two\2006-6-12_16-53-46\output\pipeline_results.h5!


,path,modification_time,creation_time,file_size
0,W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\out...,2024-09-05 07:52:47.956683,2023-08-31 22:00:50.256063,0.01
1,W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\outp...,2024-09-11 14:12:29.572562,2024-06-24 16:56:48.538135,2.06
2,W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31\out...,2024-09-04 19:59:09.388734,2023-08-03 18:14:52.394542,0.85
3,W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\out...,2024-09-04 00:01:46.173773,2023-08-03 18:15:07.920659,2.43
4,W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25\out...,2024-09-04 00:11:32.203212,2023-08-08 03:20:28.339371,1.54
5,W:\Data\KDIBA\gor01\two\2006-6-09_22-24-40\out...,2024-09-04 00:34:08.224802,2023-08-08 03:20:48.904983,2.86
6,W:\Data\KDIBA\vvp01\one\2006-4-09_17-29-30\out...,2024-06-12 10:03:53.694698,2023-08-08 03:21:51.139954,0.25
7,W:\Data\KDIBA\vvp01\one\2006-4-10_12-25-50\out...,2024-09-04 00:39:11.863333,2023-08-08 03:22:12.043931,0.75
8,W:\Data\KDIBA\vvp01\two\2006-4-09_16-40-54\out...,2024-09-04 00:41:56.797752,2023-08-08 03:22:32.722443,0.91
9,W:\Data\KDIBA\vvp01\two\2006-4-10_12-58-3\outp...,2024-09-04 00:45:10.419033,2023-08-08 03:22:53.658802,0.74


['W:\\Data\\KDIBA\\gor01\\one\\2006-6-08_14-26-15\\output\\pipeline_results.h5',
 'W:\\Data\\KDIBA\\gor01\\one\\2006-6-09_1-22-43\\output\\pipeline_results.h5',
 'W:\\Data\\KDIBA\\gor01\\one\\2006-6-12_15-55-31\\output\\pipeline_results.h5',
 'W:\\Data\\KDIBA\\gor01\\two\\2006-6-07_16-40-19\\output\\pipeline_results.h5',
 'W:\\Data\\KDIBA\\gor01\\two\\2006-6-08_21-16-25\\output\\pipeline_results.h5',
 'W:\\Data\\KDIBA\\gor01\\two\\2006-6-09_22-24-40\\output\\pipeline_results.h5',
 'W:\\Data\\KDIBA\\gor01\\two\\2006-6-12_16-53-46\\output\\pipeline_results.h5',
 'W:\\Data\\KDIBA\\vvp01\\one\\2006-4-09_17-29-30\\output\\pipeline_results.h5',
 'W:\\Data\\KDIBA\\vvp01\\one\\2006-4-10_12-25-50\\output\\pipeline_results.h5',
 'W:\\Data\\KDIBA\\vvp01\\two\\2006-4-09_16-40-54\\output\\pipeline_results.h5',
 'W:\\Data\\KDIBA\\vvp01\\two\\2006-4-10_12-58-3\\output\\pipeline_results.h5',
 'W:\\Data\\KDIBA\\pin01\\one\\11-02_17-46-44\\output\\pipeline_results.h5',
 'W:\\Data\\KDIBA\\pin01\\one\\11-

In [ ]:
## INPUTS: good_session_concrete_folders, target_dir, BATCH_DATE_TO_USE, custom_file_types_dict
from pyphoplacecellanalysis.General.Batch.runBatch import get_file_path_if_file_exists
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import copy_session_folder_files_to_target_dir

custom_file_types_dict = {'recomputed_inst_fr_comps': (lambda a_session_folder: get_file_path_if_file_exists(a_session_folder.output_folder.joinpath(f'{RESULT_DATE_TO_USE}_recomputed_inst_fr_comps_0.0005.pkl').resolve())),
						#   'PHONEW.evt': (lambda a_session_folder: get_file_path_if_file_exists(a_session_folder.output_folder.joinpath(f'{a_session_folder.context.session_name}.PHONEW.evt').resolve())),
						  }

# target_dir: Path = Path(global_data_root_parent_path)
target_dir: Path = collected_outputs_directory
moved_files_dict_files, (filelist_path, filedict_out_path) = copy_session_folder_files_to_target_dir(good_session_concrete_folders, target_dir=target_dir, RESULT_DATE_TO_USE=BATCH_DATE_TO_USE, custom_file_types_dict=custom_file_types_dict, dry_run=False)

## Extract `across_sessions_instantaneous_fr_dict` from the computation outputs

# Plotting via Plotly
`!pip install kaleido=="v0.1.0.post1" `

In [34]:
import plotly.io as pio
from pyphoplacecellanalysis.Pho2D.plotly.plotly_templates import PlotlyHelpers
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_stacked_histograms
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_helper_save_figures, _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results

# fig_size_kwargs = {'width': 1650, 'height': 480}
resolution_multiplier = 1
# fig_size_kwargs = {'width': resolution_multiplier*1650, 'height': resolution_multiplier*480}
fig_size_kwargs = {'width': resolution_multiplier*1920, 'height': resolution_multiplier*480}
is_dark_mode, template = PlotlyHelpers.get_plotly_template(is_dark_mode=False)
pio.templates.default = template

# figure_export_path = Path(r'E:\Dropbox (Personal)\Active\Kamran Diba Lab\Presentations\2024-05-30 - Pho iNAV Poster\Figures').resolve()
# figure_export_path = Path('/Users/pho/Dropbox (Personal)/Active/Kamran Diba Lab/Presentations/2024-05-30 - Pho iNAV Poster/Figures').resolve()

# assert figure_export_path.exists()

from PIL import Image

from pyphocorehelpers.programming_helpers import copy_image_to_clipboard

def save_plotly(a_fig, a_fig_context):
    """ 
    captures: TODAY_DAY_DATE, figures_folder, neptuner_run
    """
    fig_save_path: Path = figures_folder.joinpath('_'.join([TODAY_DAY_DATE, sanitize_filename_for_Windows(a_fig_context.get_description())])).resolve()
    figure_out_paths = {'.html': fig_save_path.with_suffix('.html'), '.png': fig_save_path.with_suffix('.png')}
    a_fig.write_html(figure_out_paths['.html'])
    display(fullwidth_path_widget(figure_out_paths['.html'], file_name_label='.html'))
    # print(file_uri_from_path(figure_out_paths['.html']))
    a_fig.write_image(figure_out_paths['.png'])
    # print(file_uri_from_path(figure_out_paths['.png']))
    display(fullwidth_path_widget(figure_out_paths['.png'], file_name_label='.png'))

    if neptuner_run is not None:
        a_full_figure_path_key: str = a_fig_context.get_description(separator='/', include_property_names=True, key_value_separator=':') # .replace(' ', '_')
        # a_full_figure_path_key: str = a_fig_context.get_description(separator=':', include_property_names=True, key_value_separator='|')
        print(f'a_full_figure_path_key: "{a_full_figure_path_key}"')
        # neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(a_fig)
        neptuner_run['outputs']['figures'][f"{a_full_figure_path_key}"].upload(figure_out_paths['.html'].as_posix())
        # neptuner.figures[f"{a_full_figure_path_key}"].upload(a_fig)
        
    return figure_out_paths


In [ ]:
# 'short_best_wcorr', 'long_best_wcorr', 'wcorr_diff'
# 'pearsonr_long_LR', 'long_best_wcorr', 'wcorr_diff'
all_sessions_all_scores_ripple_df

In [ ]:
concatenated_ripple_df = deepcopy(filtered_all_sessions_all_scores_ripple_df)
concatenated_ripple_df['session_name'].unique()

# ['kdiba_gor01_one_2006-6-08_14-26-15',
# 'kdiba_gor01_one_2006-6-08_14-26-15_None',
# 'kdiba_gor01_one_2006-6-09_1-22-43',
# 'kdiba_gor01_one_2006-6-12_15-55-31',
# 'kdiba_gor01_two_2006-6-07_16-40-19',
# 'kdiba_gor01_two_2006-6-08_21-16-25',
# 'kdiba_gor01_two_2006-6-09_22-24-40',
# 'kdiba_pin01_one_11-02_19-28-0',
# 'kdiba_pin01_one_fet11-01_12-58-54']

In [35]:
histogram_bins = 25
num_sessions = 1

# concatenated_ripple_df = deepcopy(df)
## INPUTS: all_sessions_all_scores_ripple_df

# concatenated_ripple_df = deepcopy(concatenated_ripple_df)
# concatenated_ripple_df = deepcopy(all_sessions_ripple_df)

# concatenated_ripple_df = deepcopy(filtered_all_sessions_wcorr_ripple_df)

concatenated_ripple_df = deepcopy(filtered_all_sessions_all_scores_ripple_df)


# concatenated_ripple_df = deepcopy(all_sessions_all_scores_ripple_df)
# concatenated_ripple_df = concatenated_ripple_df[concatenated_ripple_df['custom_replay_name'] != ''] # non-custom
# concatenated_ripple_df = deepcopy(filtered_all_sessions_all_scores_ripple_df)

# concatenated_ripple_df = deepcopy(all_sessions_ripple_time_bin_df)
# concatenated_ripple_df = deepcopy(all_sessions_all_scores_ripple_df)
# concatenated_ripple_df = deepcopy(single_time_bin_size_all_sessions_ripple_df)
display(concatenated_ripple_df)
# ['longest_sequence_length_ratio_diff', 'direction_change_bin_ratio_diff', 'congruent_dir_bins_ratio_diff', 'total_congruent_direction_change_diff']


# variable_name = 'P_Long'
variable_name = 'P_Short' # Shows expected effect - short-only replay prior to delta and then split replays post-delta
# variable_name = 'short_best_wcorr'
# variable_name = 'long_best_pf_peak_x_pearsonr'
# variable_name = 'long_best_jump'
# variable_name = 'wcorr_abs_diff'
# variable_name = 'pearsonr_abs_diff'
# variable_name = 'direction_change_bin_ratio_diff'
# variable_name = 'longest_sequence_length_ratio_diff'
# variable_name = 'long_best_longest_sequence_length_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'congruent_dir_bins_ratio_diff'
# variable_name = 'total_congruent_direction_change_diff'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'long_best_direction_change_bin_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 


# concatenated_ripple_df = concatenated_ripple_df[concatenated_ripple_df[variable_name].abs() > 0.25]

y_baseline_level: float = 0.5 # for P(short), etc
# y_baseline_level: float = 0.0 # for wcorr, etc

# px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
# px_scatter_kwargs.pop('color')


# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
hist_kwargs.pop('color')

# px_scatter_kwargs['color'] = 'custom_replay_name'
# hist_kwargs['color'] = 'custom_replay_name'


# px_scatter_kwargs['color'] = 'session_name'
# hist_kwargs['color'] = 'session_name'

px_scatter_kwargs['color'] = 'session_experience_rank'
hist_kwargs['color'] = 'session_experience_rank'

# px_scatter_kwargs['color'] = 'session_experience_orientation_rank'
# hist_kwargs['color'] = 'session_experience_orientation_rank'


new_fig_ripples, new_fig_ripples_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), legend_title_text=None, is_dark_mode=is_dark_mode)

new_fig_ripples = new_fig_ripples.update_layout(fig_size_kwargs)
_extras_output_dict = {}
if is_dark_mode:
    _extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=y_baseline_level, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
else:
    _extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=y_baseline_level, line=dict(color="rgba(0.2,0.2,0.2,.75)", width=2), row='all', col='all')

# # Update layout to add a title to the legend
# new_fig_ripples.update_layout(
#     legend_title_text='Is User Selected'  # Add a title to the legend
# )

# fig_to_clipboard(new_fig_ripples, **fig_size_kwargs)
new_fig_ripples_context = new_fig_ripples_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=new_fig_ripples_context)
new_fig_ripples

,start,stop,label,duration,is_user_annotated_epoch,is_valid_epoch,P_LR,P_RL,P_Long,P_Short,P_Long_LR,congruent_dir_bins_ratio_long_LR,coverage_long_LR,direction_change_bin_ratio_long_LR,integral_second_derivative_long_LR,jump_long_LR,longest_sequence_length_ratio_long_LR,pearsonr_long_LR,stddev_of_diff_long_LR,total_congruent_direction_change_long_LR,total_variation_long_LR,travel_long_LR,wcorr_long_LR,P_Long_RL,congruent_dir_bins_ratio_long_RL,coverage_long_RL,direction_change_bin_ratio_long_RL,integral_second_derivative_long_RL,jump_long_RL,longest_sequence_length_ratio_long_RL,pearsonr_long_RL,stddev_of_diff_long_RL,total_congruent_direction_change_long_RL,total_variation_long_RL,travel_long_RL,wcorr_long_RL,P_Short_LR,congruent_dir_bins_ratio_short_LR,coverage_short_LR,direction_change_bin_ratio_short_LR,integral_second_derivative_short_LR,jump_short_LR,longest_sequence_length_ratio_short_LR,pearsonr_short_LR,stddev_of_diff_short_LR,total_congruent_direction_change_short_LR,total_variation_short_LR,travel_short_LR,wcorr_short_LR,P_Short_RL,...,score_diff,long_best_velocity,short_best_velocity,velocity_diff,long_best_intercept,short_best_intercept,intercept_diff,long_best_speed,short_best_speed,speed_diff,long_best_wcorr,short_best_wcorr,wcorr_diff,long_best_pearsonr,short_best_pearsonr,pearsonr_diff,long_best_travel,short_best_travel,travel_diff,long_best_coverage,short_best_coverage,coverage_diff,long_best_jump,short_best_jump,jump_diff,long_best_longest_sequence_length_ratio,short_best_longest_sequence_length_ratio,longest_sequence_length_ratio_diff,long_best_direction_change_bin_ratio,short_best_direction_change_bin_ratio,direction_change_bin_ratio_diff,long_best_congruent_dir_bins_ratio,short_best_congruent_dir_bins_ratio,congruent_dir_bins_ratio_diff,long_best_total_congruent_direction_change,short_best_total_congruent_direction_change,total_congruent_direction_change_diff,long_best_total_variation,short_best_total_variation,total_variation_diff,long_best_integral_second_derivative,short_best_integral_second_derivative,integral_second_derivative_diff,long_best_stddev_of_diff,short_best_stddev_of_diff,stddev_of_diff_diff,session_experience_rank,session_experience_orientation_rank,is_novel_exposure,pre_post_delta_category
814,42.658077,42.690456,0,0.032379,False,False,0.003378,0.996622,0.968932,0.031068,0.003273,1.000000,0.385965,0.000000,0.000000,0.001919,1.000000,0.252205,0.000000,87.524594,87.524594,0.408993,0.724297,0.965659,1.000000,0.315789,0.000000,0.000000e+00,0.002503,1.000000,-0.125155,0.000000,114.162513,114.162513,0.533470,-0.515537,0.000105,1.000000,0.228070,0.000000,0.000000,0.000372,1.000000,0.178567,0.000000,11.416251,11.416251,0.079280,0.294705,0.030963,...,-0.116434,-1189.192848908021,-713.5157093448119,475.67713956320915,-50524.16480057826,-30251.71966878143,20272.445131796834,1189.192848908021,713.5157093448119,475.67713956320915,0.724297,0.294705,0.429593,0.252205,0.178567,0.073638,0.408993,0.079280,0.329714,0.385965,0.228070,0.157895,0.001919,0.000372,0.001547,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,87.524594,11.416251,76.108342,87.524594,11.416251,76.108342,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,5,2,False,pre-delta
815,55.723809,55.897899,1,0.174090,False,False,0.684715,0.315285,0.480394,0.519606,0.328933,0.333333,0.403509,0.555556,128781.306536,0.003922,0.300000,0.152431,78.542924,251.157530,490.898808,0.254880,0.173317,0.151461,0.222222,0.333333,0.333333,1.710809e+05,0.004673,0.500000,0.066632,97.482741,388.152546,570.812567,0.296372,-0.141276,0.355782,0.444444,0.280702,0.333333,16001.725371,0.003472,0.400000,0.372032,44.918969,178.854604,277.795450,0.214348,0.093641,0.163824,...,-0.032417,445.94731834045785,79.27952326052683,366.667795079931,24953.317501583566,4506.841761602025,20446.475739981543,445.94731834045785,79.27952326052683,366.667795079931,-0.141276,-0.079061,0.062216,0.066632,-0.007675,0.058957,0.296372,0.355290,-0.058918,0.

'is_user_annotated_epoch'

num_unique_sessions: 5, num_unique_time_bins: 1
num_unique_sessions: 5, num_unique_time_bins: 1
color_sequence: ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']


TypeError: plotly.express._chart_types.histogram() got multiple values for keyword argument 'color'

In [ ]:
histogram_bins = 25
num_sessions = 1

# concatenated_ripple_df = deepcopy(filtered_all_sessions_wcorr_ripple_df)
concatenated_ripple_df = deepcopy(filtered_all_sessions_simple_pearson_ripple_df) # ['P_LR', 'P_RL', 'P_Long', 'P_Short', 'ripple_idx', 'ripple_start_t', 'P_Long_LR', 'P_Long_RL', 'P_Short_LR', 'P_Short_RL', 'most_likely_decoder_index', 'start', 'stop', 'label', 'duration', 'long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'best_decoder_index', 'session_name', 'time_bin_size', 'delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch', 'custom_replay_name', 'epoch_idx', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff', 'pearsonr_abs_diff']

print(f'concatenated_ripple_df.columns: {list(concatenated_ripple_df.columns)}')
# variable_name = 'P_Long'
variable_name = 'P_Short' # Shows expected effect - short-only replay prior to delta and then split replays post-delta
# variable_name = 'short_best_wcorr'
# variable_name = 'long_best_pf_peak_x_pearsonr'
# variable_name = 'long_best_jump'
# variable_name = 'wcorr_abs_diff'
# variable_name = 'short_best_wcorr'
# variable_name = 'wcorr_long_LR'
# variable_name = 'pearsonr_abs_diff'
# variable_name = 'direction_change_bin_ratio_diff'
# variable_name = 'longest_sequence_length_ratio_diff'
# variable_name = 'long_best_longest_sequence_length_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'congruent_dir_bins_ratio_diff'
# variable_name = 'total_congruent_direction_change_diff'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'long_best_direction_change_bin_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 

y_baseline_level: float = 0.5 # for P(short), etc
# y_baseline_level: float = 0.0 # for wcorr, etc

# px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
# px_scatter_kwargs.pop('color')

# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
hist_kwargs.pop('color')
new_fig_ripples, new_fig_ripples_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), legend_title_text=None, is_dark_mode=is_dark_mode)

new_fig_ripples = new_fig_ripples.update_layout(fig_size_kwargs)

_extras_output_dict = {}
if is_dark_mode:
    _extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=y_baseline_level, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
else:
    _extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=y_baseline_level, line=dict(color="rgba(0.2,0.2,0.2,.75)", width=2), row='all', col='all')


# fig_to_clipboard(new_fig_ripples, **fig_size_kwargs)
new_fig_ripples_context = new_fig_ripples_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
# figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=new_fig_ripples_context)
new_fig_ripples

In [ ]:
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_pre_post_delta_scatter

histogram_bins = 25
num_sessions = 1

# ==================================================================================================================== #
# Laps                                                                                                                 #
# ==================================================================================================================== #
# all_sessions_laps_time_bin_df
# all_sessions_simple_pearson_laps_df

concatenated_ripple_df = deepcopy(all_sessions_simple_pearson_laps_df) # ['P_LR', 'P_RL', 'P_Long', 'P_Short', 'ripple_idx', 'ripple_start_t', 'P_Long_LR', 'P_Long_RL', 'P_Short_LR', 'P_Short_RL', 'most_likely_decoder_index', 'start', 'stop', 'label', 'duration', 'long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'best_decoder_index', 'session_name', 'time_bin_size', 'delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch', 'custom_replay_name', 'epoch_idx', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff', 'pearsonr_abs_diff']
# concatenated_ripple_df = deepcopy(all_sessions_laps_time_bin_df) # ['P_LR', 'P_RL', 'P_Long', 'P_Short', 'ripple_idx', 'ripple_start_t', 'P_Long_LR', 'P_Long_RL', 'P_Short_LR', 'P_Short_RL', 'most_likely_decoder_index', 'start', 'stop', 'label', 'duration', 'long_LR_pf_peak_x_pearsonr', 'long_RL_pf_peak_x_pearsonr', 'short_LR_pf_peak_x_pearsonr', 'short_RL_pf_peak_x_pearsonr', 'best_decoder_index', 'session_name', 'time_bin_size', 'delta_aligned_start_t', 'is_user_annotated_epoch', 'is_valid_epoch', 'custom_replay_name', 'epoch_idx', 'long_best_pf_peak_x_pearsonr', 'short_best_pf_peak_x_pearsonr', 'wcorr_long_LR', 'wcorr_long_RL', 'wcorr_short_LR', 'wcorr_short_RL', 'long_best_wcorr', 'short_best_wcorr', 'wcorr_abs_diff', 'pearsonr_abs_diff']
print(f'concatenated_ripple_df.columns: {list(concatenated_ripple_df.columns)}')
concatenated_ripple_df

# variable_name = 'P_Long'
# variable_name = 'P_Short' # Shows expected effect - short-only replay prior to delta and then split replays post-delta
variable_name = 'P_LR'

y_baseline_level: float = 0.5 # for P(short), etc
# y_baseline_level: float = 0.0 # for wcorr, etc

# px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': f"Lap Individual Time Bins - '{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
# px_scatter_kwargs['color'] = "time_bin_size"
# px_scatter_kwargs.pop('color')

hist_kwargs = dict(color="time_bin_size")
# hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
hist_kwargs.pop('color')
new_fig_laps, new_fig_laps_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), legend_title_text=None, is_dark_mode=is_dark_mode)

new_fig_laps = new_fig_laps.update_layout(fig_size_kwargs)

_extras_output_dict = {}
if is_dark_mode:
    _extras_output_dict["y_mid_line"] = new_fig_laps.add_hline(y=y_baseline_level, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
else:
    _extras_output_dict["y_mid_line"] = new_fig_laps.add_hline(y=y_baseline_level, line=dict(color="rgba(0.2,0.2,0.2,.75)", width=2), row='all', col='all')


fig_to_clipboard(new_fig_laps, **fig_size_kwargs)
new_fig_laps_context = new_fig_laps_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_fig_laps, a_fig_context=new_fig_laps_context)
new_fig_laps

In [ ]:
# variable_name = 'short_best_direction_change_bin_ratio'
# variable_name = 'long_best_congruent_dir_bins_ratio'
variable_name = 'short_best_wcorr'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': f"'{variable_name}'"} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], , 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}
# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="pre_post_delta_category") # , histnorm='probability density'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), is_dark_mode=is_dark_mode)
_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.0, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples.show()
figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=figure_context)


In [ ]:
# variable_name = 'total_congruent_direction_change_diff'
# variable_name = 'long_best_congruent_dir_bins_ratio'
# variable_name = 'long_best_total_congruent_direction_change'
variable_name = 'wcorr_diff'
# variable_name = 'long_best_wcorr'
# 'color':'is_user_annotated_epoch'
# 'color': 'is_user_annotated_epoch', 
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'color':"is_user_annotated_epoch", 'title': f"'{variable_name}'", 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size', 'is_user_annotated_epoch':'user_sel'}} # , 'color': 'time_bin_size', 'range_y': [-1.0, 1.0], 
# hist_kwargs = dict(color="time_bin_size")
hist_kwargs = dict(color="is_user_annotated_epoch") # , histnorm='probability density'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=concatenated_ripple_df, out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, hist_kwargs=hist_kwargs, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), is_dark_mode=is_dark_mode)
_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.0, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples.show()

figure_context = figure_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_fig_ripples, a_fig_context=figure_context)


## Stats Tests

In [ ]:
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import _perform_stats_tests

stats_variable_name: str = 'P_Short'
# stats_variable_name: str = 'short_best_direction_change_bin_ratio'
# stats_variable_name: str = 'short_best_wcorr'

shuffle_results, p_value, f_value, (dof1, dof2), (variance1, variance2) = _perform_stats_tests(deepcopy(concatenated_ripple_df), stats_variable_name=stats_variable_name)


In [ ]:
# Show that wcorr in both periods is higher than shuffles

stats_variable_name: str = 'P_Short'
stats_variable_name = 'short_best_wcorr'
# stats_variable_name = 'long_best_wcorr'
# stats_variable_name = 'long_best_wcorr'



shuffle_results, p_value, f_value, (dof1, dof2), (variance1, variance2) = _perform_stats_tests(deepcopy(concatenated_ripple_df), stats_variable_name=stats_variable_name)


# stats_variable_name: "short_best_wcorr" -- actual_diff_means: -0.00983910691641765
# stats_variable_name: short_best_wcorr
# Statistics=72308.00, p=0.73
# Do not Reject Null Hypothesis (No significant difference between two samples)
# Variance 1: 0.1395112660465373
# Variance 2: 0.18436187114204847
# Degree of freedom 1: 395
# Degree of freedom 2: 359
# F-statistic: 0.756725157877388
# p-value: 0.003419223265796241

# stats_variable_name: "long_best_wcorr" -- actual_diff_means: -0.0028337579937901397
# stats_variable_name: long_best_wcorr
# Statistics=71529.00, p=0.93
# Do not Reject Null Hypothesis (No significant difference between two samples)
# Variance 1: 0.1659575407149896
# Variance 2: 0.20687539745971859
# Degree of freedom 1: 395
# Degree of freedom 2: 359
# F-statistic: 0.8022101359215698
# p-value: 0.016221081810852238

In [ ]:
## User non-selected:
scatter_title = f'user_approved_ripple_df Several Sessions {variable_name}'
# variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=deepcopy(user_approved_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end), is_dark_mode=is_dark_mode)
_extras_output_dict["y_mid_line"] = new_fig_ripples.add_hline(y=0.5, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples

In [ ]:
# IDEA: The ones with clear replays (diagonal sequences in the decoded posteriors) are by definiition ambiguous, because there's not much difference between the long/short decoders.


In [ ]:
## User non-selected:
scatter_title = f'Non-selected Several Sessions {variable_name}'
# variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} # , 'color': 'time_bin_size'
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=deepcopy(user_rejected_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples


In [ ]:
# Laps test
concatenated_ripple_df = deepcopy(all_sessions_simple_pearson_laps_df)

scatter_title = 'Several Sessions'
variable_name = 'wcorr_abs_diff'
px_scatter_kwargs = {'x': 'delta_aligned_start_t', 'y': variable_name, 'title': scatter_title, 'range_y': [0.0, 1.0], 'labels': {'session_name': 'Session', 'time_bin_size': 'tbin_size'}} 
new_fig_ripples, figure_context = plotly_pre_post_delta_scatter(data_results_df=deepcopy(concatenated_ripple_df), out_scatter_fig=None, histogram_bins=histogram_bins,
                        px_scatter_kwargs=px_scatter_kwargs, histogram_variable_name=variable_name, forced_range_y=None,
                        time_delta_tuple=(earliest_delta_aligned_t_start, 0.0, latest_delta_aligned_t_end))
new_fig_ripples.update_layout(fig_size_kwargs)
new_fig_ripples


# Plotting functions

In [ ]:
from pyphoplacecellanalysis.General.Model.Configs.LongShortDisplayConfig import PlottingHelpers
# from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import plot_across_sessions_scatter_results

# Example usage:
all_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_df, concatenated_ripple_df=all_sessions_ripple_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   laps_title_prefix=f"Laps", ripple_title_prefix=f"Ripples", save_figures=True, figure_save_extension=['.html','.png'], is_dark_mode=is_dark_mode)
fig_laps, fig_ripples = all_session_figures[0]
# fig_laps.update_layout(fig_size_kwargs)
# fig_ripples.update_layout(fig_size_kwargs)

# fig_laps.show()
fig_ripples.show()
# fig_laps.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_laps.html")
# fig_ripples.write_html(f"../output/{TODAY_DAY_DATE}_AcrossSession_fig_ripples.html")


In [ ]:
figure_context = figure_context.adding_context_if_missing(num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)



In [ ]:
fig_to_clipboard(fig_ripples)

In [ ]:
## time_bin version:
all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='separate_row_per_session',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=True, figure_save_extension=['.html','.png'], is_dark_mode=is_dark_mode)
fig_time_bin_laps, fig_time_bin_ripples = all_time_bin_session_figures[0]
# fig_time_bin_laps.show()
fig_time_bin_ripples.show()

In [ ]:
## Test collapsed histograms-only results:
histograms_only_all_time_bin_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=all_sessions_laps_time_bin_df, concatenated_ripple_df=all_sessions_ripple_time_bin_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
															# enabled_time_bin_sizes=[0.03, 0.058, 0.10], # [0.03 , 0.044, 0.058, 0.072, 0.086, 0.1]
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   main_plot_mode='default',
														   laps_title_prefix=f"Laps_per_time_bin", ripple_title_prefix=f"Ripples_per_time_bin", save_figures=False, figure_save_extension=['.html','.png'])
histograms_only_fig_time_bin_laps, histograms_only_fig_time_bin_ripples = histograms_only_all_time_bin_session_figures[0]
# histograms_only_fig_time_bin_laps.show()
histograms_only_fig_time_bin_ripples.show()

In [ ]:
import plotly.subplots as sp
import plotly.graph_objs as go


all_session_figures = plot_across_sessions_scatter_results(collected_outputs_directory, concatenated_laps_df=single_time_bin_size_all_sessions_laps_df, concatenated_ripple_df=single_time_bin_size_all_sessions_ripple_df,
														#    enabled_time_bin_sizes=[0.03, 0.10],
														   earliest_delta_aligned_t_start=earliest_delta_aligned_t_start, latest_delta_aligned_t_end=latest_delta_aligned_t_end,
														   laps_title_prefix=f"Laps", ripple_title_prefix=f"Ripples", save_figures=True, figure_save_extension=['.html','.png'], is_dark_mode=is_dark_mode, main_plot_mode='default')
fig_laps, fig_ripples = all_session_figures[0]


In [ ]:
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_pre_post_delta_scatter

# data_results_df: pd.DataFrame = deepcopy(all_sessions_laps_df) # all_sessions_laps_time_bin_df
data_results_df: pd.DataFrame = deepcopy(single_time_bin_size_all_sessions_laps_time_bin_df).infer_objects()

# histogram_bins = 'auto'
histogram_bins: int = 25
new_laps_fig, new_laps_fig_context = plotly_pre_post_delta_scatter(data_results_df=data_results_df, out_scatter_fig=fig_laps, histogram_bins=histogram_bins, px_scatter_kwargs=dict(title='Laps'))


# new_laps_fig = new_laps_fig.update_layout(fig_size_kwargs, 
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     ),
# )
# Update x-axis labels
# new_laps_fig.update_xaxes(title_text="Num Time Bins", row=1, col=1)
# new_laps_fig.update_xaxes(title_text="Delta-aligned Event Time (seconds)", row=1, col=2)
# new_laps_fig.update_xaxes(title_text="Num Time Bins", row=1, col=3)


_extras_output_dict = {}
_extras_output_dict["y_mid_line"] = new_laps_fig.add_hline(y=0.5, line=dict(color="rgba(0.8,0.8,0.8,.75)", width=2), row='all', col='all')

new_laps_fig

# # Update layout to add a title to the legend
# new_fig_ripples.update_layout(
#     legend_title_text='Is User Selected'  # Add a title to the legend
# )

# fig_to_clipboard(new_fig_ripples, **fig_size_kwargs)

new_laps_fig_context: IdentifyingContext = new_laps_fig_context.adding_context_if_missing(epoch='laps', num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)
figure_out_paths = save_plotly(a_fig=new_laps_fig, a_fig_context=new_laps_fig_context)
new_laps_fig

In [ ]:
data_results_df


In [ ]:
np.unique(data_results_df['time_bin_size'])


In [ ]:

data_results_df[data_results_df['time_bin_size'] == 0.005]

# Matplotlib-based versions:

In [ ]:
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_stacked_histograms

matplotlib.use('Qt5Agg')


In [ ]:

# #TODO 2024-06-05 09:15: - [ ] Define save_matplotlib function
new_laps_fig_context: IdentifyingContext = new_laps_fig_context.adding_context_if_missing(epoch='laps', num_sessions=num_sessions, plot_type='scatter+hist', comparison='pre-post-delta', variable_name=variable_name)

In [ ]:
# from Spike3D.PendingNotebookCode import plot_stacked_histograms

# You can use it like this:'long_best_dir_simple_pearsonr'
_out0: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='Laps', session_spec='All Sessions', data_results_df=all_sessions_laps_time_bin_df, time_bin_duration_str="75 ms")
_out1: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='Ripples', session_spec='All Sessions', data_results_df=all_sessions_ripple_time_bin_df, time_bin_duration_str="75 ms")

In [ ]:


fig_to_clipboard(_out0.figures[0])

In [ ]:
## all_sessions_new_laps_df, all_sessions_new_ripple_df
_out0: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='New Laps', session_spec='All Sessions', data_results_df=all_sessions_new_laps_df, time_bin_duration_str="25 ms")
_out1: "MatplotlibRenderPlots" = plot_histograms_across_sessions(data_type='New Ripples', session_spec='All Sessions', data_results_df=all_sessions_new_ripple_df, time_bin_duration_str="25 ms")



In [ ]:
all_sessions_new_laps_df

In [ ]:
len(all_sessions_new_laps_df['session_name'].unique()) # 10 sessions



In [ ]:

matplotlib.use('Qt5Agg')

column_name: str = 'P_Short'

# You can use it like this:
num_unique_sessions: int = all_sessions_laps_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_laps_time_bin_df.time_bin_size.nunique(dropna=True)
_laps_histogram_out = plot_stacked_histograms(all_sessions_laps_time_bin_df, 'Laps', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes", column_name=column_name)

fig_to_clipboard(_laps_histogram_out.figures[0])

num_unique_sessions: int = all_sessions_ripple_time_bin_df.session_name.nunique(dropna=True) # number of unique sessions, ignoring the NA entries
num_unique_time_bins: int = all_sessions_ripple_time_bin_df.time_bin_size.nunique(dropna=True)
_ripple_histogram_out = plot_stacked_histograms(all_sessions_ripple_time_bin_df, 'Ripples', f'{num_unique_sessions} Sessions', f"{num_unique_time_bins} tbin sizes", column_name=column_name)



In [ ]:
all_sessions_ripple_time_bin_df['time_bin_size'].unique()

In [ ]:
# active_time_bin_size: float = 0.072 # 0.058
# active_time_bin_size: float = 0.086
# active_time_bin_size: float = 0.1 # looks the most bimodal with as little of an intermediate value as possible.
active_time_bin_size: float = 0.13444444 # looks the most bimodal with as little of an intermediate value as possible.
_single_time_bin_ripple_histogram_out = plot_stacked_histograms(all_sessions_ripple_time_bin_df[np.isclose(all_sessions_ripple_time_bin_df['time_bin_size'], active_time_bin_size)], 'Ripples', f'{num_unique_sessions} Sessions', f"tbin: {active_time_bin_size:0.3f}s", column_name=column_name)
fig_to_clipboard(_single_time_bin_ripple_histogram_out.figures[0], bbox_inches='tight')


In [ ]:
fig_to_clipboard(_ripple_histogram_out.figures[0])